# MemoryBanditWorkflow を使った RAG の試験実装 (LangChain 1.x系)

(Version: 0.0.16.1)

## 概要

熊剣迷路問題の経験を元に作ったエージェントフレームワークの MemoryBanditWorkflow。それを LangChain 1.x 系に対応させ、さらにはやりの skills やツールボックスなどの導入に相当する subtool_do の導入も行った。

それは迷路問題の一部として作り始めたが、汎用なものになるようこころがけた。それが実際汎用に使えることを示すために、RAG エージェントを作ってみたのが今回の試みである。

## RagAgent のアイデア

MemoryBanditWorkflow のアイデアとサブツールのアイデアについては、↓をご参照いただきたい。

《langchain_maze_0_0_15.ipynb - JRF-2018/langchain_maze》  
https://github.com/JRF-2018/langchain_maze/blob/master/langchain_maze_0_0_15.ipynb

基本的にはメモリ機能とバンディット機能とワークフロー機能を持っているのが MemoryBanditWorkflow で、今回の物はそれをそのまま使って、RagAgent という子クラスを定義している。

なお、MemoryBanditWorkflow はセマンティックサーチとかを用意するのは、めんどくさかったので、そういうバックエンドはこれまた AI さんに「データベース偽装」をしてもらっている。

MemoryBanditWorkflow の元となった迷路問題は↓からたどっていただきたい。

《JRF-2018/langchain_maze: 熊剣迷路問題 revisited》  
https://github.com/JRF-2018/langchain_maze

ところで、RAG エージェントといえばマルチエージェントを使うのが流行りのようだが、今回の物はリニアに順次、シングルスレッドで実行されていくだけである。マルチエージェントを敷衍するなら asyncio を使ったり、メモリにデータベースを使ったりすべきなのだが、そういうことはできる展望すらない。あくまで MemoryBanditWorkflow で RAG も実装できることを示そうとしたものなので、RAG としての新規性はない。

## 前回へのリンク

《experimental_rag_0_0_2.ipynb - JRF-2018/langchain_maze》  
https://github.com/JRF-2018/langchain_maze/blob/master/experimental_rag_0_0_2.ipynb

## 前回からの変更点

LangChain 1.x 系に対応した。まず、Pydantic v2 と Gemini の実装のクセらしく型エラーへの対処が難しかったが、なんとか今は動くようになっている。しかし、弥縫策で、この先はどうなるか自信がない。

ツールを格納して必要なツールのみ説明を読んで使うための subtool_do や subtool_show を導入した。subtool_show で表示されるのがおよそ他のフレームワークでは SKILL.md で書かれることに相当する。


## 結論

gemini-2.5-flash-lite さんでテストし、最後は gemini-3-flash-preview さんにお願した。途中、こちらのミスによる無限ループがわかったところで、一度止めて修正して再実行したそのログをそのまま載せている。コストがかかるのでそうした。ご容赦いただきたい。

サブツールの実行はやはり、難しいらしく、/thesis サブツールを使っての完成は一発ではならなかった。そのサブツールを使うようなじったあとは、完成までこぎつけることができた。

ただ、それらの影響かどうか、内容は別として、構造としては前回のほうがよかったように思う。でも、MemoryBanditWorkflow とサブツールの実証実験としてはこんなものだと思う。

なお、再現性がないため、最初からの再実行はやっていないのはご容赦願いたい。コストもかかるしね \(^^;。


## 著者

JRF ( http://jrf.cocolog-nifty.com/statuses , Twitter (X): @jion_rockford )


## ライセンス

基本短いコードなので(私が作った部分は)パブリックドメインのつもりです。気になる方は MIT License として扱ってください。

かなり AI さん達(Gemini さんや ChatGPT さんや Claude さんや Grok さん)に教わって作っています。


## 実装

まず、必要なライブラリを読み込みます。

In [13]:
!pip install -q -U langchain langchain-google-genai duckduckgo-search langchain-community beautifulsoup4 ddgs


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 6.7 MB/s eta 0:00:00


Gemini にアクセスします。シークレットで Gemini API キーを Google AI Studio からインポートすると GOOGLE_API_KEY というシークレットができるはずです。それを使います。

In [21]:
import os
from langchain.chat_models import init_chat_model
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from google.colab import userdata

#os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

llm = init_chat_model(
    "google_genai:gemini-3-flash-preview",
    google_api_key=userdata.get('GOOGLE_API_KEY'),
#    thinking_level="low", # for gemini-3.0
#    thinking_budget=0, # for gemini-2.5
)
emb_llm = GoogleGenerativeAIEmbeddings(
    model='gemini-embedding-001',
    google_api_key=userdata.get('GOOGLE_API_KEY'),
)


ちゃんと Gemini にアクセスできるかテストします。

In [22]:
import os
from langchain_core.messages import HumanMessage

# Gemini 3 から次のようなヘルパー関数が必要となった。
def get_content_text(content):
    if isinstance(content, list):
        return "".join([item.get('text', '') for item in content if item.get('type') == 'text'])
    return content

response = llm.invoke([HumanMessage(content="Geminiモデルの特徴を教えてください")])
print(get_content_text(response.content))


Googleが開発した「Gemini（ジェミニ）」は、従来のAIモデルとは一線を画す、非常に強力で汎用性の高いAIモデルです。

その主な特徴を5つのポイントで解説します。

### 1. ネイティブ・マルチモーダル
Geminiの最大の特徴は、最初から**「マルチモーダル」**として設計・学習されている点です。
*   **多様な入出力:** テキストだけでなく、画像、音声、動画、プログラミングコードを同時に理解し、処理することができます。
*   **シームレスな理解:** 例えば、「動画を見せて、その中で起きている出来事を要約させ、さらにその背景にある物理法則を解説させる」といった、異なる種類の情報を組み合わせた高度な推論が得意です。

### 2. 圧倒的なコンテキストウィンドウ（情報の処理容量）
Gemini 1.5 Proなどのモデルでは、一度に処理できる情報の量（コンテキストウィンドウ）が非常に大きいです。
*   **最大200万トークン:** これは、数千ページの文書、数時間の動画、あるいは膨大な量のソースコードを一度に読み込めることを意味します。
*   **メリット:** 長い資料の中から特定の情報を探し出したり（「 haystack（干し草の山）」テストで高い精度を誇る）、大規模なプロジェクト全体のコードを理解したりすることが可能です。

### 3. 用途に合わせたモデル展開
Geminiは、利用シーンに合わせて最適化された複数のサイズ（モデル）が用意されています。
*   **Gemini Ultra:** 最も複雑なタスク向け。高度な推論、科学、コーディングに特化。
*   **Gemini Pro:** 幅広いタスクに対応する万能型。パフォーマンスと速度のバランスが良い。
*   **Gemini Flash:** 応答速度とコスト効率を重視。リアルタイム性が求められるアプリに向いている。
*   **Gemini Nano:** デバイス（スマホなど）上での実行に特化。オフラインでも動作し、プライバシー保護に優れる（Pixel 8/9などに搭載）。

### 4. Googleエコシステムとの強力な連携
Googleの各種サービスと深く統合されているため、実用性が非常に高いです。
*   **Google検索との連携:** 常に最新

埋め込みベクトルも試しておきます。

In [4]:
emb_llm.embed_query("これはテストです。")[:5]

[-0.0140901245, -0.006826138, 0.0076896483, -0.07491117, 0.011200756]

基本的なモジュールを読み込みます。

In [5]:
import os
import math
import numpy as np
import random
import re
from pprint import pprint
from time import sleep
import pickle
np.set_printoptions(legacy='1.25')

セーブ／ロードのために次のコードを実行します。

In [6]:
RAG_AGENT_SAVE = "rag-agent.pickle"

ライブラリ集。

In [7]:
from pydantic import ValidationError
from typing import List, Dict, Any, Tuple, Union
from textwrap import dedent
import datetime
import copy
import inspect
from IPython.display import Markdown

# LangChainのコンポーネントをインポート
from langchain_core.tools import tool, Tool
from langchain.agents.middleware import SummarizationMiddleware
from langchain.agents.middleware.summarization import DEFAULT_SUMMARY_PROMPT
from langchain.agents import create_agent
#from langgraph.prebuilt import create_react_agent
#from langchain_core.messages.utils import count_tokens_approximately
#from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.prompts.chat import ChatPromptTemplate
#from langmem.short_term import SummarizationNode, summarize_messages
from langchain_core.messages import AIMessage, ToolMessage, HumanMessage, SystemMessage
from langgraph.errors import GraphRecursionError
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.document_loaders import WebBaseLoader

SUMMARY_PROMPT = DEFAULT_SUMMARY_PROMPT + "\n\n**要約は日本語でしてください。**"


In [8]:
def calc_embedding_variance(embeddings):
    if not embeddings or len(embeddings) < 2:
        return 0.0

    embeddings_array = np.array(embeddings)
    mean_vector = np.mean(embeddings_array, axis=0)
    squared_distances = np.linalg.norm(embeddings_array - mean_vector, axis=1)**2
    variance = np.mean(squared_distances)

    return variance

def short_repr(x, max_len=80):
    repr_str = repr(x)

    if len(repr_str) > max_len:
        ellipsis_len = 3

        head_len = max_len - ellipsis_len - 1
        tail_len = 1
        return repr_str[:head_len] + "..." + repr_str[-tail_len:]
    else:
        return repr_str

def get_content_text(content):
    if isinstance(content, list):
        return "".join([item.get('text', '') for item in content if item.get('type') == 'text'])
    return content


MemoryBanditWorkflow は langchain_maze_0.0.15.ipynb と変わっていないです。長大な定義で申し訳ないです。

In [27]:
class MemoryBanditWorkflow:
    def __init__ (self, llm=llm, llm2=llm, emb_llm=emb_llm,
                  save_file=None):
        self.llm = llm
        self.llm2 = llm2
        self.emb_llm = emb_llm
        self.save_file = save_file

        self.core_context = ""
        self.plan = "まだ計画と方針はセットされていません。"
        self.scratchpad = ""

        self.messages = []
        self.running_summary = None
        self.system_prompt = """\
これはメモリ機能とバンディット機能の動きを見るための実験です。
現在の計画と方針と周囲の状況を考慮し、必要に応じて計画と方針を更新してください。
あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。
実験にふさわしく、できる限り、ツールを利用し、特に、メモリを検索し、その文書を更新して、後のコマンドに備えてください。
メモリの id は memory:～ という形式です。memory:5555 を指定するときは 5555 だけではダメです。文の中でメモリの参照を示すときは [memory:〜] のように書きます。
「メモリの文書を検索する手順」は [memory:9998] にあります。「メモリに文書を残す手順」は [memory:9997] にあります。
ツールの多くはサブツールとして格納されています。サブツール名は "/dir1/subtool1" のように "/" から始まる名前をしています。どういうサブツールがあるか知るためにまずは subtool_show("/") を実行してください。
"""

        self.backend_status = None
        self.backend_result = None
        self.messages2 = []
        self.system_prompt2 = """\
エージェントをサポートする裏方のエージェントです。
本来この裏方は様々な技法を用いて実装される予定なのですが、現在は試験実装中のため、その動きをあなたは偽装しなければなりません。

よく考えツールを積極的に使って Human からの指示に従ってください。
"""

        self.memories = {}
        self.keywords = []

        self.tools = {}
        self.tool_tag = "null_tools"

        self.access_unit = 1.0
        self.recent_reads = []

        self.workflows = {}
        self.workflow_current = "workflow:main"
        self.workflow_next = None
        self.privileged_tool_names = []

        self.init_memories()
        self.init_workflows()
        self.init_tools()


    def __getstate__ (self):
        state = self.__dict__.copy()
        del state['llm']
        del state['llm2']
        del state['emb_llm']
        del state['tools']
        #del state['agent']
        return state

    def __setstate__ (self, state):
        self.__dict__.update(state)
        self.prev_load = True

    def save (self):
        if not self.save_file:
            return
        with open(self.save_file, 'wb') as f:
            pickle.dump(self, f)

    @classmethod
    def load (cls, filename, llm=llm, llm2=llm, emb_llm=emb_llm):
        with open(filename, 'rb') as f:
            loaded_game = pickle.load(f)
        loaded_game.llm = llm
        loaded_game.llm2 = llm2
        loaded_game.emb_llm = emb_llm
        loaded_game.tools = {}
        loaded_game.init_tools()
        return loaded_game

    def normalize_memory_id(self, id_or_num):
        if isinstance(id_or_num, int):
            return f"memory:{id_or_num}"
        elif isinstance(id_or_num, str):
            m = re.search(r'\[?memory:(\d+)\]?', id_or_num)
            if m:
                return f"memory:{m.group(1)}"
            if id_or_num.isdigit():
                return f"memory:{id_or_num}"
            else:
                return id_or_num
        else:
            return id_or_num

    def _normalize_workflow_id_sub(self, id_or_num):
        if isinstance(id_or_num, int):
            return f"workflow:{id_or_num}"
        if id_or_num in ["current", "main"]:
            return f"workflow:{id_or_num}"
        elif isinstance(id_or_num, str):
            m = re.search(r'\[?workflow:(\d+|main|current)\]?(?:.+)?', id_or_num.strip())
            if m:
                return f"workflow:{m.group(1)}"
            if id_or_num.isdigit():
                return f"workflow:{id_or_num}"
            else:
                return id_or_num
        else:
            return id_or_num

    def normalize_workflow_id(self, id_or_num):
        r = self._normalize_workflow_id_sub(id_or_num)
        if r == "workflow:current":
            return self.workflow_current
        return r

    def register_tool (self, tool, tags=None):
        if not tags:
            tags = ["default_tools", "all_tools"]
        self.tools[tool.name] = {
            'name': tool.name,
            'tags': tags,
            'tool': tool
        }

    def change_tool_tags (self, tool, tags=None):
        if not tags:
            tags = ["default_tools", "all_tools"]
        name = tool if isinstance(tool, str) else tool.name
        self.tools[name]['tags'] = tags

    def register_subtools (self, directory, subtools,
                           description=None, content=None,
                           tags=None):
        if not tags:
            tags = ["default_tools", "all_tools"]
        assert directory.startswith("/")
        if directory not in self.tools:
            self.tools[directory] = {
                'name': directory,
            }
        if description:
            self.tools[directory]['description'] = description
        if content:
            self.tools[directory]['content'] = content

        # 初期設定時は content と description が必要
        assert 'description' in self.tools[directory]
        assert 'content' in self.tools[directory]

        for name, tool in subtools:
            assert name.startswith(directory + "/")
            self.tools[name] = {
                'name': name,
                'tags': tags,
                'tool': tool,
            }

    def _create_tool_manual(self, tool):
        tool_name = tool.name
        tool_description = getattr(tool, "description", "説明はありません。")

        arg_names = []
        if hasattr(tool, "args_schema") and tool.args_schema:
            arg_names = list(tool.args_schema.__fields__.keys())
        else:
            func = getattr(t1, "func", tool)
            sig = inspect.signature(func)
            arg_names = [p for p in sig.parameters.keys() if p != 'self']

        args_str = ", ".join(arg_names)
        args_dict_str = ", ".join([f'"{name}": ...' for name in arg_names])

        manual = f"""\
【ツール名】 {tool_name}
【書式】 {tool_name}({args_str})
【説明】 {tool_description}
"""
        return manual

    def _create_subtool_manual(self, subtool_name, tool):
        import inspect
        tool_name = tool.name
        tool_description = getattr(tool, "description", "説明はありません。")

        arg_names = []
        if hasattr(tool, "args_schema") and tool.args_schema:
            arg_names = list(tool.args_schema.model_fields.keys())
        else:
            func = getattr(t1, "func", tool)
            sig = inspect.signature(func)
            arg_names = [p for p in sig.parameters.keys() if p != 'self']

        args_str = ", ".join(arg_names)
        args_dict_str = ", ".join([f'"{name}": ...' for name in arg_names])

        manual = f"""\
【サブツール名】 {subtool_name}
【元のツール名】 {tool_name}
【元の書式】 {tool_name}({args_str})
【説明】 {tool_description}

※このツールを実行するには、直接呼び出すのではなく、必ず以下の subtool_do を使用してください：
【正しい書式】subtool_do("{subtool_name}", {{{args_dict_str}}})
"""
        return manual

    def create_tool_skill (self, name):
        if name == "/":
            r = dedent("""\
            ---
            name: /
            description: サブツールのルート。サブツールの検索の仕方を説明する。
            allowed-tools: サブツールを使うのに特別な許可は必要ありません。
            ---

            サブツールはディレクトリ(サブスキル)に分かれて入っています。

            各サブスキルに入っているサブツールを見るには、例えばツール subtool_show("/sys") を実行します。そこには SKILL.md のような説明が入っています。

            ## サブスキル

            """)
            for dir in self.tools:
                if "description" in self.tools[dir]:
                    e = self.tools[dir]
                    r += f"-  **{e['name']}**: {e['description']}\n"
            return r

        name = name.rstrip("/")
        if name not in self.tools:
            return None
        e = self.tools[name]
        if "tool" in e:
            if "content" in e:
                r = dedent(f"""\
                ---
                name: {e['name']}
                description: {e['description']}
                allowed-tools: このサブツールを使うのに特別な許可は必要ありません。
                ---
                """)
                r += e['content']
                return r
            if e['name'].startswith("/"):
                manual = self._create_subtool_manual(e['name'], e['tool'])
            else:
                manual = self._create_tool_manual(e['tool'])
            if self.tool_tag in e['tags']:
                manual += "【現在のコンテキストにおいて】使用できます。\n"
            else:
                manual += "【現在のコンテキストにおいて】使用できません。\n"
            r = dedent(f"""\
            ---
            name: {e['name']}
            description: {e['tool'].name}
            allowed-tools: このサブツールを使うのに特別な許可は必要ありません。

            ---
            """)
            r += manual
            return r

        r = dedent(f"""\
        ---
        name: {e['name']}
        description: {e['description']}
        allowed-tools: このサブツールを使うのに特別な許可は必要ありません。
        ---
        """)
        r += e['content']

        dirs = [name for name, x in self.tools.items()
                if name.startswith(e['name'] + "/")
                and 'description' in x]
        subtools = [name for name, x in self.tools.items()
                    if name.startswith(e['name'] + "/")
                    and 'description' not in x]

        if dirs:
            r += "\n## サブスキル\n\n"
            for dir in dirs:
                x = self.tools[dir]
                r += f"-  **{x['name']}**: {x['description']}\n"

        if subtools:
            r += "\n## サブツール\n\n"
            for subtool_name in subtools:
                x = self.tools[subtool_name]
                manual = self._create_subtool_manual(x['name'], x['tool'])
                r += dedent(f"""\

                ### サブツール: {x['name']}

                """)
                r += manual

        return r

    def _replace_tools (self, from_tools, to_tools):
        tool_names = [x.name for x in to_tools]
        return [x for x in from_tools
                if x.name not in tool_names] + to_tools

    def init_tools (self):
        @tool
        def express_thought (thought: str) -> None:
            """
            プレイヤーの現在の考えを吐露します。
            """
            mes = f"「{thought}」と考えが吐露されました。"
            print(f"ツール(express_thought): {mes}")

        @tool
        def update_plan (new_plan: str) -> str:
            """
            プレイヤーの現在の計画と方針を更新します。
            表示されるべき新しい計画と方針の文字列を提供してください。
            あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。
            """
            self.plan = new_plan
            mes = "計画と方針が更新されました。"
            print(f"ツール(update_plan): {mes}: {new_plan}")
            return mes

        @tool
        def show_core () -> str:
            """
            コアコンテクストを返します。
            """
            print(f"ツール(show_plan): {self.core_context}")
            return self.core_context

        @tool
        def update_core (new_core: str) -> str:
            """
            コアコンテクストを更新します。
            表示されるべき新しいコアコンテクストの文字列を提供してください。
            コアコンテクストとは要約機能などのあとなどコンテクストが失われたあとにも思い出すべき情報(memory_read や subtool_show すべきものなど)です。
            """
            self.core_context = new_core
            mes = "コアコンテクストが更新されました。"
            print(f"ツール(update_plan): {mes}: {new_core}")
            return mes

        @tool
        def show_plan () -> str:
            """
            プレイヤーの現在の計画と方針を返します。
            """
            print(f"ツール(show_plan): {self.plan}")
            return self.plan

        @tool
        def update_scratchpad (new_scratchpad: str) -> str:
            """
            自由に使えるスクラッチパッドを更新します。
            """
            self.scratchpad = new_scratchpad
            mes = "スクラッチパッドが更新されました。"
            print(f"ツール(update_scratchpad): {mes}: {new_scratchpad}")
            return mes

        @tool
        def show_scratchpad () -> str:
            """
            スクラッチパッドを返します。
            """
            print(f"ツール(show_scratchpad): {self.scratchpad}")
            return self.scratchpad

        @tool
        def memory_new (title: str, text: str) -> str:
            """
            指定された title と text によるメモリを構成します。

            Returns:
                str: 割り当てられた memory_id。
            """

            i = 1000
            while True:

                if f"memory:{i}" not in self.memories:
                    break
                i = i + 1
            new_id = f"memory:{i}"
            self.memories[new_id] = {
                'id': new_id,
                'title': title,
                'accesses': 0,
                'text': text,
                'modified_at': datetime.datetime.now().isoformat()
            }
            self.update_keywords(text)
            self.update_vector(self.memories[new_id])
            print(f"ツール(memory_new): {short_repr(self.memories[new_id])}")
            return new_id

        @tool
        def memory_update_string (
                memory_id: str,
                from_str: str,
                to_str: str
        ) -> str:
            """
            指定されたmemory_idの記憶の中にある文字列を修正します。

            Args:
                memory_id (str): 修正する記憶のID。
                from_str (str): 置換元の文字列を含む文字列。
                to_str (str): 置換先の文字列を含む文字列。

            Returns:
                str: 処理結果を説明する簡潔なメッセージ。
            """

            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"

            original_title = self.memories[memory_id]['title']
            original_text = self.memories[memory_id]['text']

            if from_str not in original_text and from_str not in original_title:
                return f"エラー: 置換元の文字列 '{from_str}' が記憶内に見つかりませんでした。"

            updated_title = original_title.replace(from_str, to_str)
            updated_text = original_text.replace(from_str, to_str)

            self.memories[memory_id]['title'] = updated_title
            self.memories[memory_id]['text'] = updated_text
            self.memories[memory_id]['modified_at'] = datetime.datetime.now().isoformat()
            self.update_keywords(updated_text)
            self.update_vector(self.memories[memory_id])

            return f"成功: 記憶ID '{memory_id}' の文字列を '{from_str}' から '{to_str}' に修正しました。"

        @tool
        def memory_append_string (
                memory_id: str,
                string_to_append: str,
                separator: str = '\n'
        ) -> str:
            """
            指定されたmemory_idの記憶に文字列を追記します。
            """
            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"


            original_text = self.memories[memory_id]['text']
            updated_text = original_text + separator + string_to_append
            self.memories[memory_id]['text'] = updated_text
            self.memories[memory_id]['modified_at'] = datetime.datetime.now().isoformat()
            self.update_keywords(updated_text)
            self.update_vector(self.memories[memory_id])

            return f"成功: 記憶ID '{memory_id}' に文字列 '{string_to_append}' を追記しました。"

        @tool
        def memory_delete (memory_id: str) -> str:
            """
            指定されたmemory_idの記憶に文字列を追記します。
            """
            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"

            del self.memories[memory_id]

            return f"成功: 記憶ID '{memory_id}' を削除しました。"

        @tool
        def memory_read(memory_id: str) -> Union[Dict[str, str], str]:
            """
            指定されたIDの記憶を読み込みます。

            Args:
                memory_id (str): 読み込む記憶のID。

            Returns:
                Union[Dict[str, str], str]: 記憶が見つかった場合は辞書、
                      見つからない場合はエラーメッセージ文字列を返します。
            """

            memory_id = self.normalize_memory_id(memory_id)
            if memory_id in self.memories:
                self.memories[memory_id]['accesses'] += self.access_unit * 1.0
                self.recent_reads.append(self.memories[memory_id])
                self.recent_reads = self.recent_reads[-10:]
                r = self.memories[memory_id].copy()
                del r['vector']
                return r
            else:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

        @tool
        def memory_list_recent(top_n: int = 10) -> Dict[str, Any]:
            """
            最近変更されたメモリを新しい順でリストします。
            """

            filter_date = datetime.datetime(2025, 1, 1)
            sorted_memories = sorted(
                [memory for memory in self.memories.values()
                 if datetime.datetime.fromisoformat(memory['modified_at'])
                 >= filter_date],
                key=lambda x: datetime.datetime.fromisoformat(x['modified_at']),
                reverse=True
            )
            if sorted_memories:
                if len(sorted_memories) > top_n:
                    sorted_memories = sorted_memories[:top_n]
                r = [{'id': x['id'], 'title': x['title'],
                      'modified_at': x['modified_at']}
                     for x in sorted_memories]
                return {'status': 'success', 'result': r}
            else:
                return {'status': 'error',
                        'result': 'エラー: 最近のメモリはありません。'}

        @tool
        def memory_list_random(top_n: int = 10) -> Dict[str, Any]:
            """
            メモリをランダムにリストします。
            """

            keys = list(self.memories.keys())
            if len(keys) > top_n:
                keys = random.sample(keys, top_n)
            if keys:
                values = [self.memories[k] for k in keys]
                r = [{'id': x['id'], 'title': x['title'],
                      'modified_at': x['modified_at']}
                     for x in values]
                return {'status': 'success', 'result': r}
            else:
                return {'status': 'error',
                        'result': 'エラー: メモリはありません。'}

        @tool
        def memory_words_search(search_str: str) -> Dict[str, Any]:
            """
            メモリ内を search_str で文字列検索します。OR や () が使えます。
            """

            res = self.call_backend_agent(dedent(f"""\
            メモリ全部を search_str = {repr(search_str)} で文字列検索するのを偽装してください。OR や () が使えることになっています。

            ただし、メモリやキーワードは read_all_memories や read_all_keywords ツールで得られる本物のメモリやキーワードを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「マッチデータ」のリスト。
            マッチデータはそれぞれが辞書型、それを m とすると。
            m['id'] はメモリ id。memory:〜 という形式。
            m['title'] はメモリの title。
            m['snippet'] はメモリの text のその文字列にマッチする部分周辺。
            """))
            if res['status'] == 'success':
                for m in res['result']:
                    if 'id' in m and m['id'] in self.memories:
                        x = self.memories[m['id']]
                        x['accesses'] += self.access_unit * 0.1
            return res

        @tool
        def memory_semantic_search(search_str: str) -> Dict[str, Any]:
            """
            メモリ内を search_str でセマンティックサーチします。
            """

            res = self.call_backend_agent(dedent(f"""\
            メモリ全部を search_str = {repr(search_str)} でセマンティックにサーチするのを偽装してください。

            ただし、メモリは read_all_memories や ツールで得られる本物のメモリを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「マッチデータ」のリスト。
            マッチデータはそれぞれが辞書型、それを m とすると。
            m['id'] はメモリ id。memory:〜 という形式。
            m['title'] はメモリの title。
            m['snippet'] はメモリの text のそのセマンティックにマッチする部分周辺。
            """))
            if res['status'] == 'success':
                for m in res['result']:
                    if 'id' in m and m['id'] in self.memories:
                        x = self.memories[m['id']]
                        x['accesses'] += self.access_unit * 0.1
            return res

        @tool
        def imagine_keywords(thought: str) -> List[Tuple[str, float]]:
            """
            thought からキーワードをスコア付きで連想します。
            """

            r = self.call_backend_agent(dedent(f"""\
            thought = {repr(thought)} からキーワードをスコア付きで複数連想してください。

            ただし、キーワードは read_all_memories や read_all_keywords ツールで得られる本物のキーワードを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「キーワードデータ」のリスト。
            キーワードデータは文字列とスコアからなるタプル。
            """))
            if r['status'] == 'success':
                return r["result"]
            else:
                return []

        @tool
        def bandit_schedule(tool_name: str, times: int, prob: float, exec_mode: str = "persistent", aux_prompt: str = "", workflow_id: str = "workflow:current") -> str:
            """
            ツールの利用を強制するためのバンディットを予約します。

            Args:
                tool_name: 強制するツールの名前。" OR " で区切って複数登録できる。
                times: 一度で何回同じ物を登録するか。times == 0 にもできます。
                prob: 一回の実行がある確率。prob == 0.0 にもできます。
                exec_mode: "once" or "persistent"
                aux_prompt: 実行の詳細を示すための補助プロンプト または ""
                workflow_id: バンディットを予約するワークフローを指定します。

            Returns:
                str: "成功" または "失敗" とその理由を返します。
            """

            tool_names = re.split(r"\s+or\s+|\s+OR\s+", tool_name)
            prohibited = set(self.privileged_tool_names) & set(tool_names)
            if prohibited:
                return f"失敗。{repr(prohibited)} は登録できません。"
            all_tools = [name for name, x in self.tools.items()
                         if "tool" in x]
            if not any (x in all_tools for x in tool_names):
                return f"失敗。{tool_name} は有効なツールでありません。"

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if 'w' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は書き換え不能のワークフローです。"

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == tool_name \
                   and x['exec_mode'] == exec_mode \
                   and x['aux_prompt'] == aux_prompt \
                   and x['arg'] is None:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
                    if times == 0 or prob == 0.0:
                        return "成功。バンディトを削除しました。"
                    self.workflows[workflow_id]['stack'].append(x)
            else:
                if times == 0 or prob == 0.0:
                    return "失敗。そのようなバンディットはありません。バンディットを指定するには tool_name exec_mode aux_prompt のすべてを一致させて指定する必要があります。"
                x = {
                    'pin': 'stack' if exec_mode != "once" else None,
                    'arg': None
                }
                self.workflows[workflow_id]['stack'].append(x)
            if x['pin'] == "write":
                return f"失敗。'{tool_name}' は書き換えられません。"
            else:
                x['tool_name'] = tool_name
                x['tools_name'] = 'default_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = aux_prompt
                x['prob'] = prob
                x['times'] = times
                print(f"ツール(bandit_schedule): {repr(x)}")
                if dest is None:
                    return "成功。バンディットを登録しました。"
                else:
                    return "成功。バンディットを更新しました。"

        @tool
        def bandit_schedule_memory_read(memory_id: str, times: int, prob: float, exec_mode: str = "persistent", workflow_id: str = "workflow:current") -> str:
            """
            memory_read ツールの利用を強制するためのバンディットを予約します。

            Args:
                memory_id: memory_read するための memory_id。
                times: 一度で何回同じ物を登録するか。times == 0 にもできます。
                prob: 一回の実行がある確率。prob == 0.0 にもできます。
                exec_mode: "once" or "persistent"
                workflow_id: バンディットを予約するワークフローを指定します。

            Returns:
                str: "成功" または "失敗" とその理由を返します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if 'w' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は書き換え不能のワークフローです。"

            memory_id = self.normalize_memory_id(memory_id)

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == "memory_read" \
                   and x['exec_mode'] == exec_mode \
                   and not x['aux_prompt'] \
                   and x['arg'] == memory_id:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
                    if times == 0 or prob == 0.0:
                        return "成功。バンディトを削除しました。"
                    self.workflows[workflow_id]['stack'].append(x)
            else:
                if times == 0 or prob == 0.0:
                    return "失敗。そのようなバンディットはありません。バンディットを指定するには exec_mode memory_id のすべてを一致させて指定する必要があります。"
                x = {'pin': None, 'arg': memory_id}
                self.workflows[workflow_id]['stack'].append(x)
            if x['pin'] == "write":
                return f"失敗。'memory_read {memory_id}' は書き換えられません。"
            else:
                x['tool_name'] = 'memory_read'
                x['tools_name'] = 'read_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = ""
                x['prob'] = prob
                x['times'] = times
                print(f"ツール(bandit_schedule_memory_read): {repr(x)}")
                if dest is None:
                    return "成功。バンディットを登録しました。"
                else:
                    return "成功。バンディットを更新しました。"

        @tool
        def bandit_schedule_subtool_show(subtool_name: str, times: int, prob: float, exec_mode: str = "persistent", workflow_id: str = "workflow:current") -> str:
            """
            subtool_show ツールの利用を強制するためのバンディットを予約します。

            Args:
                subtool_name: subtool_show するための subtool_name。
                times: 一度で何回同じ物を登録するか。times == 0 にもできます。
                prob: 一回の実行がある確率。prob == 0.0 にもできます。
                exec_mode: "once" or "persistent"
                workflow_id: バンディットを予約するワークフローを指定します。

            Returns:
                str: "成功" または "失敗" とその理由を返します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if 'w' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は書き換え不能のワークフローです。"

            if subtool_name not in self.tools:
                return f"失敗。{subtool_name} は有効な名前ではありません。"

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == "subtool_show" \
                   and x['exec_mode'] == exec_mode \
                   and not x['aux_prompt'] \
                   and x['arg'] == subtool_name:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
                    if times == 0 or prob == 0.0:
                        return "成功。バンディトを削除しました。"
                    self.workflows[workflow_id]['stack'].append(x)
            else:
                if times == 0 or prob == 0.0:
                    return "失敗。そのようなバンディットはありません。バンディットを指定するには exec_mode subtool_name のすべてを一致させて指定する必要があります。"
                x = {'pin': None, 'arg': subtool_name}
                self.workflows[workflow_id]['stack'].append(x)
            if x['pin'] == "write":
                return f"失敗。'subtool_show {subtool_name}' は書き換えられません。"
            else:
                x['tool_name'] = 'subtool_show'
                x['tools_name'] = 'read_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = ""
                x['prob'] = prob
                x['times'] = times
                print(f"ツール(bandit_schedule_subtool_show): {repr(x)}")
                if dest is None:
                    return "成功。バンディットを登録しました。"
                else:
                    return "成功。バンディットを更新しました。"

        @tool
        def bandit_schedule_workflow(workflow_id_to_schedule: str, times: int, prob: float, exec_mode: str = "persistent", workflow_id: str = "workflow:current") -> str:
            """
            workflow_do ツールの利用を強制するためのバンディットを予約します。

            Args:
                workflow_id_to_schedule: workflow_do するための workflow_id。
                times: 一度で何回同じ物を登録するか。times == 0 にもできます。
                prob: 一回の実行がある確率。prob == 0.0 にもできます。
                exec_mode: "once" or "persistent"
                workflow_id: バンディットを予約するワークフローを指定します。

            Returns:
                str: "成功" または "失敗" とその理由を返します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if 'w' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は書き換え不能のワークフローです。"

            workflow_id2 = self.normalize_workflow_id(workflow_id_to_schedule)
            if workflow_id2 not in self.workflows:
                return f"失敗。{workflow_id2} は有効なワークフローではありません。"

            dest = None
            for i, x in enumerate(self.workflows[workflow_id]['stack']):
                if x['tool_name'] == "workflow_do" \
                   and x['exec_mode'] == exec_mode \
                   and not x['aux_prompt'] \
                   and x['arg'] == workflow_id2:
                    dest = i
                    break
            if dest is not None:
                x = self.workflows[workflow_id]['stack'][dest]
                if not x['pin']:
                    self.workflows[workflow_id]['stack'].pop(dest)
                    if times == 0 or prob == 0.0:
                        return "成功。バンディトを削除しました。"
                    self.workflows[workflow_id]['stack'].append(x)
            else:
                if times == 0 or prob == 0.0:
                    return "失敗。そのようなバンディットはありません。バンディットを指定するには exec_mode workflow_id_to_schedule のすべてを一致させて指定する必要があります。"
                x = {
                    'pin': 'stack' if exec_mode != "once" else None,
                    'arg': workflow_id2
                }
                self.workflows[workflow_id]['stack'].append(x)
            if x['pin'] == "write":
                return f"失敗。'workflow_do {workflow_id2}' は書き換えられません。"
            else:
                x['tool_name'] = 'workflow_do'
                x['tools_name'] = 'default_tools'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = ""
                x['prob'] = prob
                x['times'] = times
                print(f"ツール(bandit_schedule_workflow): {repr(x)}")
                if dest is None:
                    return "成功。バンディットを登録しました。"
                else:
                    return "成功。バンディットを更新しました。"

        @tool
        def bandit_list(workflow_id: str =  "workflow:current")  -> Dict[str, Any]:
            """
            登録されているバンディットのスタックを返します。

            Args:
                workflow_id: バンディットのスタックが登録されたワークフローを指定します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            return {'status': 'success',
                    'result': self.workflows[workflow_id]['stack']}

        @tool
        def bandit_statistics()  -> str:
            """
            バンディットに有用かもしれない統計情報を返します。
            """
            s_read = calc_embedding_variance([
                x['vector'] for x in self.recent_reads
            ])
            s_write = calc_embedding_variance([
                x['vector'] for x in self.memories.values()
            ])
            accesses = [x['accesses'] for x in self.memories.values()]
            accesses.sort()
            accesses = accesses[:len(accesses) // 2]
            if accesses:
                s_access = np.mean(accesses)
            else:
                s_access = 0.0

            return dedent(f"""\
            最近 10個のメモリ read の分散: {s_read}
            メモリの分散: {s_write}
            下位50%のアクセス数の平均: {s_access}
            """)

        @tool
        def subwork_done()  -> str:
            """
            指示されたサブワークが完了したことを宣言します。サブワークが完了したときのみ使ってください。
            """

            return f"成功。サブワークの完了が宣言されました。"

        @tool
        def workflow_do(workflow_id: str)  -> str:
            """
            ワークフローを実行します。
            """

            if self.workflow_next:
                return f"失敗。すでに {self.workflow_next} の実行が予約中です。"
            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} は有効なワークフローではありません。"
            if 'e' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は子として実行できません。"
            self.workflow_next = workflow_id
            title = self.workflows[workflow_id]['title']
            return f"成功。この後、{workflow_id} 「{title}」を実行していきます。"

        @tool
        def workflow_list()  -> Dict[str, Any]:
            """
            登録されているすべてのワークフローの id とその title および pin 状況を返します。
            """

            return {'status': 'success',
                    'result': list(self.workflows.values())}

        @tool
        def workflow_show_current()  -> str:
            """
            現在実行中のワークフローのタイトル、および、強制しているバンディットの情報を表示します。
            """

            wtitle = self.workflows[self.workflow_current]['title']
            mes = dedent(f"""\
            現在のワークフロー: {self.workflow_current} 「{wtitle}」
            現在のバンディットの指示: 「{self.cur_bandit_prompt}」
            現在のバンディット: {repr(self.cur_bandit)}
            現在のバンディットの done 数: {self.cur_bandit_done}
            """)
            print(f"ツール(workflow_show_current): {mes}")
            return mes


        @tool
        def workflow_new(title: str, bandits: List[Dict[str, Any]], pin: str)  -> str:
            """
            新しいワークフローを定義し、その workflow_id を返します。

            Args:
                title: ワークフローの名前。
                bandits: 登録するバンディットのリスト。
                pin: 'w' は書換不能。'd' は削除不能。'wd' は書換＆削除不能。'' は書換＆削除可能。

            バンディットはそれぞれ辞書型、それを b とすると。
            b['tool_name'] は bandit_schedule での tool_name と同じ。
            b['exec_mode'] は bandit_schedule の exec_mode と同じ。
            b['aux_prompt'] は bandit_schedule の aux_prompt と同じ。
            b['prob'] は bandit_schedule の prob と同じ。
            b['times'] は bandit_schedule の times と同じ。
            b['arg'] は b['tool_name'] が 'memory_read' のとき memory:〜 という memory_id を指定できる。または、b['tool_name'] が 'workflow_do' のとき workflow:〜 という workflow_id を指定できる。
            b['pin'] は、None のとき削除または prob と times の書き換え可能。'stack' のときは prob と times の書き換え可能。'write' のときは書き換えも不能。
            """
            # AI に対する隠し属性として pin の 'e' は子として実行不能。

            for b in bandits:
                if not all(x in b for x in ['tool_name', 'exec_mode',
                                            'aux_prompt', 'times', 'prob']):
                    return f"失敗。有効な定義ではありません。"
                if 'arg' not in b:
                    b['arg'] = None
                if b['tool_name'] not in ["memory_read", "workflow_do"] \
                   and b['arg']:
                    return f"失敗。有効な定義ではありません。"
                if b['arg'] and b['tool_name'] == "memory_read":
                    b['tools_name'] = "read_tools"
                else:
                    b['tools_name'] = "default_tools"
                if 'pin' not in b:
                    b['pin'] = None
                if not (b['pin'] is None or b['pin'] == 'stack'
                        or b['pin'] == 'write'):
                    return f"失敗。不正な pin 値です。"
                tool_names = re.split(r"\s+or\s+|\s+OR\s+", b['tool_name'])
                prohibited = set(self.privileged_tool_names) & set(tool_names)
                if prohibited:
                    return f"失敗。{repr(prohibited)} は登録できません。"
                all_tools = [name for name, x in self.tools.items()
                                 if "tool" in x and b['tools_name'] in x.tags]
                if not any (x in all_tools for x in tool_names):
                    return f"失敗。{b['tool_name']} は有効なツールでありません。"

            i = 1000
            while True:

                if f"workflow:{i}" not in self.workflows:
                    break
                i = i + 1
            new_id = f"workflow:{i}"

            self.workflows[new_id] = {'stack': bandits, 'pin': pin,
                                      'title': title}
            print(f"ツール(workflow_new): {repr(self.workflows[new_id])}")
            return f"成功。{new_id} を新規登録しました。"

        @tool
        def workflow_delete(workflow_id: str)  -> str:
            """
            ワークフローを削除します。
            """

            workflow_id = self.normalize_workflow_id(workflow_id)
            if workflow_id not in self.workflows:
                return f"失敗。{workflow_id} はありません。"
            if 'd' in self.workflows[workflow_id]['pin']:
                return f"失敗。{workflow_id} は削除できません。"
            del self.workflows[workflow_id]
            return f"成功。{workflow_id} を削除しました。"

        @tool
        def subtool_show(subtool_name: str)  -> str:
            """
            subtool_name で表されたディレクトリまたはサブツールのスキルを返します。
            """

            r = self.create_tool_skill(subtool_name)
            if r:
                return r
            else:
                return f"エラー: {subtool_name} がないか、スキルを構成できません。"

        @tool
        def subtool_do(subtool_name: str, args_dict: Dict[str, Any])  -> Any:
            """
            subtool_name というサブツールを実行します。

            例えば、"/sys/tool1" という名前の元のツールが t1 だとして、def t1(arg1, arg2) と定義されていたとき、t1("a", "b") を呼び出すには subtool_do("/sys/tool1", {"arg1": "a", "arg2": "b"}) などとして呼び出します。


            Args:
                subtool_name (str): "/" で始まるサブツール名。
                args_dict (dict): 引数を表す辞書。

            """
            if subtool_name not in self.tools:
                return f"エラー: {subtool_name} というサブツールはありません。"
            if 'tool' not in self.tools[subtool_name]:
                return f"エラー: {subtool_name} は実行可能なツールではありません。もしかすると subtool_show('{subtool_name}')が必要なのかも。"
            if self.tool_tag not in self.tools[subtool_name]['tags']:
                return f"エラー: {subtool_name} は現在実行可能なツールではありません。コンテクストにより実行できるツールは違います。"
            target_tool = self.tools[subtool_name]['tool']

            try:
                if hasattr(target_tool, "args_schema") and target_tool.args_schema:
                    target_tool.args_schema.model_validate(args_dict)

                if hasattr(target_tool, "invoke"):
                    result = target_tool.invoke(args_dict)
                else:
                    result = target_tool.run(args_dict)

                return result
            except ValidationError as e:
                error_details = e.errors()
                return f"エラー: 引数の形式が正しくありません。\n詳細: {error_details}"

        main_tools = [
            express_thought,
            update_scratchpad, show_scratchpad,
            memory_read, memory_list_recent, memory_list_random,
            memory_semantic_search, memory_words_search,
            imagine_keywords,
            subwork_done,
            workflow_do,
            subtool_show, subtool_do,
        ]
        sys_tools = [
            update_core, show_core,
            update_plan, show_plan,
            bandit_schedule, bandit_schedule_memory_read, bandit_list,
            bandit_statistics,
            workflow_new, workflow_list,
            workflow_show_current, workflow_delete,
            bandit_schedule_workflow,
            bandit_schedule_subtool_show,
        ]
        write_tools = [
            memory_new, memory_update_string, memory_append_string,
            memory_delete,
        ]

        for t in main_tools + write_tools:
            self.register_tool(t, tags=["default_tools", "read_tools",
                                        "all_tools"])
        for t in write_tools:
            self.change_tool_tags(t, tags=["default_tools", "all_tools"])
        sys_subtools = [(f"/sys/{t.name}", t) for t in sys_tools]
        self.register_subtools(
            directory="/sys",
            subtools=sys_subtools,
            description="システムに基本的なサブツール集",
            content=dedent("""\
            これらはシステムで基本的に使えるサブツール集です。
            """),
            tags=["default_tools", "read_tools", "all_tools"]
        )


    def _create_agent (self, tools_name='default_tools'):
        self.tool_tag = tools_name
        tools = []
        for name in self.tools:
            if not name.startswith("/"):
                x = self.tools[name]
                if self.tool_tag in x["tags"]:
                    tools.append(x["tool"])

        summarizer = SummarizationMiddleware(
            model=self.llm,
            trigger=("tokens", 5000),
            keep=("messages", 20),
            summary_prompt=SUMMARY_PROMPT,
        )

        app = create_agent(
            model=self.llm, tools=tools, system_prompt=self.system_prompt,
            middleware=[summarizer],
            checkpointer=InMemorySaver(), name="main-agent",
        )

        return app

    def _filterout_messages2(self):
        self.messages = [
            x for x in self.messages
            if x.id not in self.messages2ids
        ]

    def _sanitize_messages(self):
        print("おかしなエラーが出ているため対症療法として messages をサニタイズします。")
        self.messages = [
            m for m in self.messages
            if not (isinstance(m, AIMessage) and m.tool_calls)
        ]

    def run (self, workflow_main_id):
        print("\n\n----------\n\n")
        self.messages2ids = []

        self.workflow_current = workflow_main_id
        bandits = copy.deepcopy(
            self.workflows[self.workflow_current]['stack']
        )
        arg1s = {}
        working_bandit = None
        workflow_stack = []
        execed = []
        while True:
            while working_bandit is not None or bandits:
                if working_bandit is not None:
                    b, done, prev_done = working_bandit
                    working_bandit = None
                else:
                    b = bandits.pop()
                    done = 0
                    prev_done = True
                enforce = b['tool_name']
                aux_prompt = b['aux_prompt']
                tools_name = b['tools_name']
                memory_id = None
                workflow_id = None
                subtool_show_name = None
                if b['arg'] and enforce == 'memory_read':
                    memory_id = b['arg']
                if b['arg'] and enforce == 'workflow_do':
                    workflow_id = b['arg']
                if b['arg'] and enforce == 'subtool_show':
                    subtool_show_name = b['arg']

                while done < b['times']:
                    if not random.random() < b['prob']:
                        done += 1
                        continue
                    if memory_id and memory_id not in self.memories:
                        done += 1
                        continue
                    if workflow_id and workflow_id not in self.workflows:
                        done += 1
                        continue
                    all_tools = [name for name, x in self.tools.items()
                                 if "tool" in x]
                    tool_names = re.split(r"\s+or\s+|\s+OR\s+", enforce)
                    if not any (x in all_tools for x in tool_names):
                        done += 1
                        continue
                    if memory_id:
                        aux_prompt = f"{memory_id} を読んでください。"
                    if workflow_id:
                        aux_prompt = f"{workflow_id} を実行してください。"
                    self.cur_bandit = b
                    if subtool_show_name:
                        aux_prompt = f"{subtool_show_name} を読んでください。"
                    self.cur_bandit_done = done
                    self.cur_bandit_prompt = f"補助にツールをいろいろ使いながら最終的に {enforce} {'のどれか' if ' or ' in enforce.lower() else ''}を適当なパラメータで使ってください。{'(補助プロンプト): ' + aux_prompt if aux_prompt else ''}"
                    prompt = self.cur_bandit_prompt
                    if not prev_done:
                        prompt = "前回の指示がまだ完了してません。前回の指示: " + prompt
                    print(f"USER_INPUT: {prompt}")
                    self.messages.append(HumanMessage(prompt))
                    config = {"configurable": {"thread_id": "1"},
                              "recursion_limit": 25}
                    app = self._create_agent(tools_name=tools_name)
                    self.access_unit = 0.3 if memory_id else 1.0
                    prev_done = False
                    self.workflow_next = None
                    app_stream = None
                    try:
                        for chunk0 in app.stream(
                                {"messages": self.messages.copy()},
                                config=config,
                                stream_mode="updates",
                        ):
                            self.messages = app.get_state(config).values["messages"].copy()
                            if 'model' in chunk0:
                                for chunk in chunk0['model']['messages']:
                                    if hasattr(chunk, "tool_calls") \
                                       and chunk.tool_calls:
                                        for tool_call in chunk.tool_calls:
                                            t_id = tool_call.get('id')
                                            args = tool_call.get('args', {})
                                            if tool_call["name"] == 'subtool_do':
                                                arg1s[t_id] = args.get('subtool_name')
                                            elif tool_call["name"] == 'subtool_show':
                                                arg1s[t_id] = args.get('subtool_name')
                                            elif tool_call["name"] == 'memory_read':
                                                arg1s[t_id] = self.normalize_memory_id(args.get('memory_id'))
                                            elif tool_call["name"] == 'workflow_do':
                                                arg1s[t_id] = self.normalize_workflow_id(args.get('workflow_id'))
                            if 'tools' not in chunk0:
                                continue
                            done2 = 0
                            for chunk in chunk0['tools']['messages']:
                                if chunk.id in self.messages2ids:
                                    print("!WHY!")
                                    continue
                                if not isinstance(chunk, ToolMessage):
                                    continue
                                last_tool = chunk.name
                                arg1 = None
                                if last_tool == 'subtool_do':
                                    last_tool = arg1s.get(chunk.tool_call_id, "!UNKNOWN!")
                                    if not last_tool.startswith("/"):
                                         last_tool = chunk.name
                                if last_tool in ['memory_read', 'subtool_show', 'workflow_do']:
                                    arg1 = arg1s.get(chunk.tool_call_id, "!UNKNOWN!")
                                print(f"ツール結果({last_tool}): {short_repr(chunk.content)}", flush=True)

                                if last_tool == "workflow_do":
                                    if last_tool in re.split(r"\s+or\s+|\s+OR\s+", enforce) \
                                       and (not workflow_id or workflow_id == self.workflow_next):
                                        done += 1
                                        prev_done = True
                                        execed.append(b)
                                        if not self.workflow_next:
                                            done2 = 1
                                            break
                                    if not self.workflow_next:
                                        continue
                                    workflow_stack.append((
                                        (b, done, prev_done),
                                        bandits,
                                        execed,
                                        self.workflow_current
                                    ))
                                    self.workflow_current = self.workflow_next
                                    bandits = copy.deepcopy(self.workflows[self.workflow_current]['stack'])
                                    working_bandit = None
                                    execed = []
                                    done2 = 1
                                    break
                                elif last_tool in re.split(r"\s+or\s+|\s+OR\s+", enforce) \
                                   and (not memory_id or memory_id == arg1) \
                                   and (not subtool_show_name or subtool_show_name == arg1):
                                    done += 1
                                    prev_done = True
                                    execed.append(b)
                                    done2 = 1
                                    break
                            if done2:
                                break
                        self._filterout_messages2()
                        #self._summarize_messages()
                        print(f"エージェントの応答: {get_content_text(self.messages[-1].content)}")
                    except GraphRecursionError as e:
                        print(f"Recursion Limit に到達しました。")
                        self._filterout_messages2()
                        #self._summarize_messages()
                    except Exception as e:
                        print(f"エラーが発生しました(main): {e}")
                        import traceback
                        traceback.print_exc()
                        self._sanitize_messages()
                        raise e

            for b in execed:
                for x in self.workflows[self.workflow_current]['stack']:
                    if x['tool_name'] == b['tool_name'] \
                       and x['exec_mode'] == b['exec_mode'] \
                       and x['aux_prompt'] == b['aux_prompt'] \
                       and x['arg'] == b['arg'] \
                       and x['exec_mode'] == "once":
                        if x['times'] > 0:
                            x['times'] -= 1
            self.workflows[self.workflow_current]['stack'] = [
                x for x in self.workflows[self.workflow_current]['stack']
                if x['exec_mode'] != 'once' or x['pin'] or x['times'] > 0
            ]

            if not workflow_stack:
                break
            workflow_prev = self.workflow_current
            prev_title = self.workflows[workflow_prev]['title']
            working_bandit, bandits, execed, self.workflow_current \
                = workflow_stack.pop()
            cur_title = self.workflows[self.workflow_current]['title']
            mes = f"{workflow_prev} 「{prev_title}」から {self.workflow_current} 「{cur_title}」に復帰しました。"
            print(f"USER_INPUT: {mes}")
            self.messages.append(HumanMessage(mes))

    def listen_and_print (self, prompt):
        ans = None
        try:
            app = self._create_agent(tools_name='null_tools')
            config = {"configurable": {"thread_id": "1"}}
            print(f"USER_INPUT: {prompt}")
            response = app.invoke(
                {"messages": self.messages + [HumanMessage(prompt)]},
                config=config
            )
            self.messages = response['messages']
            #self._summarize_messages()
            ans = get_content_text(response['messages'][-1].content)
            print(f"エージェントの応答: {ans}")
        except Exception as e:
            print(f"エラーが発生しました(listen_and_print): {e}")
            raise e
        print("")
        sleep(3)
        return ans

    def init_memories (self):
        memories = [
            {
                'id': 'memory:9998',
                'title': 'メモリの文書を検索する手順',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                まずどういうものが探したいか、express_thought してみる。

                その上でそれにまつわるキーワードを imagine_keywords で連想してみる。

                それらにしたがって memory_words_search や memory_semantic_search をやってみる。
                """)
            },
            {
                'id': 'memory:9997',
                'title': 'メモリに文書を残す手順',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                行動結果や得た知識を積極的にメモリに書き残していこう。

                メモリに書くときは次のような要素を使う。

                [memory:〜] : メモリID への参照を明示する。
                keyword:〜 : そのメモリにまつわるキーワードを指定する。

                キーワードは将来のメモリへも実質的にリンクできることに注意しよう。

                例:

                私が [memory:5555] に従って歩いていると確かに妖怪に出くわした。

                keyword: 妖怪

                おそろしかった。
                """)
            },
            {
                'id': 'memory:9995',
                'title': 'ツールが実行できない！と思ったら',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                指示に無関係なツールは実行できないことがある。現在使えるツールをよく確かめよう。
                """)
            },
            {
                'id': 'memory:9994',
                'title': 'keyword の増強',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                memory_list_random という機能を使って 5 つリストアップして、それぞれを読み、それぞれに適切なキーワードを付けることが可能なら 「keyword: 〜」文を append するように。
                """)
            }
        ]
        for x in memories:
            self.update_keywords(x['text'])
            self.memories[x['id']] = x
            self.update_vector(x)

    def init_workflows (self):
        workflow_main = [
            {
                'tool_name': 'memory_new',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "最近のやり取りを要約して書いてください。",
                'arg': None,
                'prob': 0.1,
                'times': 1,
                'pin': 'stack'
            },
            # {
            #     'tool_name': 'subwork_done',
            #     'tools_name': 'default_tools',
            #     'exec_mode': 'persistent',
            #     'aux_prompt': "ワークフローのテストをします。適当なパラメータで workflow_new を実行し、それをすぐさま workflow_delete します。そして、bandit_schedule で、指示しているこのバンディットを削除します。完了したら subwork_done してください。",
            #     'arg': None,
            #     'prob': 1.0,
            #     'times': 1,
            #     'pin': None,
            # },
            {
                'tool_name': 'memory_new OR memory_update_string OR memory_append_string',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': None,
                'prob': 0.4,
                'times': 1,
                'pin': 'stack'
            },
            {
                'tool_name': 'workflow_do',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': "workflow:1000",
                'prob': 1.0/20,
                'times': 1,
                'pin': 'stack'
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': None,
                'prob': 0.5,
                'times': 3,
                'pin': 'stack'
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9998',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9997',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
        ]
        self.workflows["workflow:main"] \
            = {'pin': 'de', 'stack': workflow_main, 'title': "メイン"}

        workflow_sub = [
            {
                'tool_name': 'subwork_done',
                'tools_name': 'default_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "memory:9994 を読みその指示を実行し、そのタスクが終ったら、subwork_done してください。",
                'arg': None,
                'prob': 1.0,
                'times': 1,
                'pin': 'write'
            },
            {
                'tool_name': 'memory_read',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9994',
                'prob': 1.0,
                'times': 1,
                'pin': 'write'
            }
        ]
        self.workflows["workflow:1000"] \
            = {'pin': 'wd', 'stack': workflow_sub, 'title': 'keyword 更新'}

    def update_keywords (self, text):
        extracted_keywords = []

        pattern1 = r'keyword:\s*(.*?)(?:\n|$)'
        matches1 = re.findall(pattern1, text, re.IGNORECASE)
        extracted_keywords.extend([kw.strip() for kw in matches1])

        pattern2 = r'\[keyword:\s*(.*?)\]'
        matches2 = re.findall(pattern2, text, re.IGNORECASE)
        extracted_keywords.extend([kw.strip() for kw in matches2])

        for keyword in extracted_keywords:
            if keyword.startswith("〜"):
                continue
            if keyword and keyword not in self.keywords:
                self.keywords.append(keyword)

    def update_vector (self, x):
        text = x['title'] + "\n\n" + x['text']
        x['vector'] = self.emb_llm.embed_query(text)

    def _create_backend_agent (self):
        @tool
        def set_result (status: str, res: Union[Dict, List, str, int, float, bool, None])  -> None:
            """
            結果を設定します。

            Args:
            status (str): 処理のステータス ('success' または 'error')。
            res: 結果データ（リスト、辞書、数値、文字列など何でも可）
            """
            print(f"ツール2(set_result): status: {repr(status)}, result: {short_repr(res)}")
            self.backend_status = status
            self.backend_result = res

        @tool
        def read_all_memories ()  -> Dict[str, Any]:
            """
            全ての記憶を読み込みます。
            """
            print("ツール2(read_all_memories):...")
            return {'status': 'success',
                    'result': [{k: v for k, v in x.items() if k != 'vector'}
                               for x in self.memories.values()]}

        @tool
        def read_all_keywords ()  -> Dict[str, Any]:
            """
            全てのキーワードを読み込みます。
            """
            print("ツール2(read_all_keywords):...")
            return {'status': 'success',
                    'result': [x for x in self.keywords]}

        @tool
        def express_thought (thought: str) -> None:
            """
            プレイヤーの現在の考えを吐露します。
            """
            mes = f"「{thought}」と考えが吐露されました。"
            print(f"ツール2(express_thought): {mes}")

        tools = [set_result, read_all_memories, read_all_keywords,
                 express_thought]

        app = create_agent(
            model=self.llm2, tools=tools, system_prompt=self.system_prompt2,
            checkpointer=InMemorySaver(), name="sub-agent",
        )

        return app

    def call_backend_agent (self, user_input):
        config = {"configurable": {"thread_id": "2"}}
        app = self._create_backend_agent()
        self.messages2 = []
        self.backend_result = None
        self.backend_status = None

        while self.backend_result is None or self.backend_status is None:
            try:
                sleep(3)
                print(f"USER_INPUT2: {user_input}")
                self.messages2.append(HumanMessage(user_input))
                for chunk0 in app.stream(
                        {"messages": self.messages2.copy()},
                        config=config,
                        stream_mode="updates",
                        name="sub-agent",
                ):
                    self.messages2 = app.get_state(config).values["messages"].copy()
                    done = 0
                    if "tools" not in chunk0:
                        continue
                    for x in chunk0['tools']['messages']:
                        self.messages2ids.append(x.id)
                        if isinstance(x, ToolMessage):
                            print(f"ツール結果2({x.name}): {short_repr(x.content)}", flush=True)
                        if isinstance(x, ToolMessage) and x.name == "set_result":
                            done = 1
                            break
                    if done:
                        break
                print(f"エージェントの応答: {get_content_text(self.messages2[-1].content)}")
            except GraphRecursionError as e:
                print(f"Recursion Limit に到達しました。")
            except Exception as e:
                print(f"エラーが発生しました(sub): {e}")
                import traceback
                traceback.print_exc()
                raise e

            sleep(3)

        return {'status': self.backend_status, 'result': self.backend_result}


メインの RagAgent。

In [35]:
class RagAgent (MemoryBanditWorkflow):
    def __init__ (self, llm=llm, llm2=llm, emb_llm=emb_llm,
                  save_file=None):
        self.thesis = {
            'title': "",
            'chapters': [{'title': '概要', 'text': ""}]
        }
        super().__init__(llm=llm, llm2=llm, emb_llm=emb_llm,
                         save_file=save_file)

        self.system_prompt = """\
あなたは賢い RAG エージェントです。論文(/thesis)を書き上げていきます。
利用可能なツールを使用して、迷路をナビゲートし、ゴールに到達してください。
現在の計画と方針と周囲の状況を考慮し、必要に応じて計画と方針を更新してください。
あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。

メモリの id は memory:～ という形式です。memory:5555 を指定するときは 5555 だけではダメです。文の中でメモリの参照を示すときは [memory:〜] のように書きます。
「全地図と地図記号」は [memory:9999] にあります。「メモリの文書を検索する手順」は [memory:9998] にあります。「メモリに文書を残す手順」は [memory:9997] にあります。

ツールの多くはサブツールとして格納されています。サブツール名は "/dir1/subtool1" のように "/" から始まる名前をしています。どういうサブツールがあるか知るためにまずは subtool_show("/") を実行してください。

論文(/thesis)は章(chapter)に分けて作っていきます。それぞれが Markdown 形式です。論文のタイトルは # 一つ、章のタイトルは ## で修飾されることになりますので、それに合わせた階層で書いてください。論文の中に [memory:〜] という内部メモリの参照を入れてはいけません。ですからメモリに蓄えるときに参照 URL などは別途記録しておいてください。
"""

        self.system_prompt2 = """\
あなたは賢い RAG エージェントをサポートする裏方のエージェントです。
本来この裏方は様々な技法を用いて実装される予定なのですが、現在は試験実装中のため、その動きをあなたは偽装しなければなりません。

よく考えツールを積極的に使って Human からの指示に従ってください。
"""
        self.subwork_done = True
        self.user_demand = ''
        self.current_work = ''
        self.current_state = 0

    def init_tools (self):
        super().init_tools()

        @tool
        def show_user_demand () -> str:
            """
            大目的を返します。
            """
            print(f"ツール(show_user_demand): {self.user_demand}")
            return self.user_demand

        @tool
        def show_current_work () -> str:
            """
            中目的すなわち現在の仕事を返します。
            """
            print(f"ツール(show_current_work): {self.current_work}")
            return self.current_work

        @tool
        def thesis_write_title (new_title: str) -> str:
            """
            論文のタイトルを更新します。
            """
            self.thesis['title'] = new_title
            mes = "論文のタイトルが更新されました。"
            print(f"ツール(thesis_write_title): {mes}: {new_title}")
            return mes

        @tool
        def thesis_show_title () -> str:
            """
            論文のタイトルを返します。
            """
            print(f"ツール(thesis_show_title): {self.thesis['title']}")
            return self.thesis['title']

        @tool
        def thesis_new_chapter (title: str, text: str) -> str:
            """
            新しい章を title と text で構成します。
            """

            x = {'title': title, 'text': text}
            num = len(self.thesis['chapters'])
            self.thesis['chapters'].append(x)
            print(f"ツール(thesis_new_chapter): {short_repr(x)}")
            return f"成功: 第{num}章を作りました。"

        @tool
        def thesis_write_chapter (
                chapter_num: int,
                new_title: str,
                new_text: str
        ) -> str:
            """
            章を new_title と new_text で置き換えます。
            """

            if not (chapter_num >= 0
                    and chapter_num < len(self.thesis['chapters'])):
                return f"失敗: 第{chapter_num}章は現在ありません。"
            x = {'title': new_title, 'text': new_text}
            self.thesis['chapters'][chapter_num] = x
            print(f"ツール(thesis_write_chapter): {short_repr(x)}")
            return f"成功: 第{chapter_num}章を書き換えました。"

        @tool
        def thesis_delete_last_chapter () -> str:
            """
            最後の章を削除します。
            """
            num = len(self.thesis['chapters']) - 1
            if num <= 0:
                return f"失敗: これ以上削除できません。"
            self.thesis['chapters'].pop()
            return f"成功: 第{num}章を削除しました。"

        @tool
        def thesis_read_chapter(chapter_num: int) -> Union[Dict[str, str], str]:
            """
            指定された章を読み込みます。
            """

            if not (chapter_num >= 0
                    and chapter_num < len(self.thesis['chapters'])):
                return f"失敗: 第{chapter_num}章は現在ありません。"
            return self.thesis['chapters'][chapter_num]

        @tool
        def thesis_list_chapters() -> List[str]:
            """
            章のタイトルのリストを返します。
            """
            return [x['title'] for x in self.thesis['chapters']]

        @tool
        def subwork_done()  -> str:
            """
            指示されたサブワークが(成功裡に)完了したことを宣言します。
            """
            self.subwork_done = True
            return f"成功。サブワークの完了が宣言されました。"

        @tool
        def subwork_not_done()  -> str:
            """
            指示されたサブワークが完了してないことを宣言します。
            """
            self.subwork_done = False
            return f"成功。サブワークの未完了が宣言されました。"

        @tool
        def show_execution_map () -> str:
            """
            workflow より上位の実行制御の全体像とそこにおける現在の State を返します。
            """
            mes = dedent(f"""\
            大目的は「{self.user_demand}」です。

            次に実行制御の全体像を示します。State 0 からはじまります。指示があったとき subwork_done または subwork_not_done することで次の State に遷移します。

            State 0: 大目的に沿ってメモリ等に計画を立てます。完了時 State 1 へ。

            State 1: 大目的に沿ってウェブを調査し取材してまだ書いていない一章分ずつメモリ等に記録していきます。完了時 State 2 へ。

            State 2: 大目的に沿ってメモリ等にある取材データが全章分、十分か確かめます。完了時 State 3 へ。未完了時 State 1 へ。

            State 3: 大目的に沿ってメモリ等にある取材データを元に論文(/thesis)をまだ書いてない一章分、十分でない一章分を書きます。完了時 State 4 へ。

            State 4: 大目的に沿って取材データを元にした論文(/thesis)が全章完成しているかをチェックします。最後にタイトルを設定し、第0章の概要を書き直します。完了時 State 6 へ。未完了時 State 5 へ。

            State 5: 大目的に沿ってまだ追加取材が必要なら再調査してメモリ等に記録します。完了時 State 3 へ

            State 6: 終了。完成！

            現在の State は {self.current_state} です。
            """)
            print(f"ツール(show_execution_map): {mes}")
            return mes


        @tool
        def get_web_page_content(url: str) -> str:
            """
            指定されたURLからWebページのコンテンツを取得して文字列として返します。
            調査でURLを見つけた後に使ってください。
            """
            try:
                loader = WebBaseLoader(url)
                docs = loader.load()
                content = " ".join([doc.page_content for doc in docs])
                print(f"ツール(get_web_page_content): 成功: {url}")
                # 長すぎる場合は適当に丸める
                if len(content) > 5000:
                    return content[:5000] + "... (コンテンツが長すぎるため省略)"
                return content
            except Exception as e:
                print(f"ツール(get_web_page_content): 失敗: {url}")
                return f"エラー: ページの読み込みに失敗しました。URL: {url}, エラー: {e}"

        search_tool = DuckDuckGoSearchResults(output_format="list", verbose=True)
        web_page_tool = get_web_page_content

        main_tools = [subwork_done, subwork_not_done, search_tool, web_page_tool]
        for t in main_tools:
            self.register_tool(t, tags=["default_tools", "read_tools",
                                        "all_tools"])
        thesis_subtools = [
            ('/thesis/write_title', thesis_write_title),
            ('/thesis/show_title', thesis_show_title),
            ('/thesis/new_chapter', thesis_new_chapter),
            ('/thesis/write_chapter', thesis_write_chapter),
            ('/thesis/delete_last_chapter', thesis_delete_last_chapter),
            ('/thesis/read_chapter', thesis_read_chapter),
            ('/thesis/list_chapters', thesis_list_chapters),
        ]
        self.register_subtools(
            directory="/thesis",
            subtools=thesis_subtools,
            description="論文を書くためのサブツール",
            content=dedent("""\
            これらは論文を書くためのサブツール集です。

            論文(/thesis)は章(chapter)に分けて作っていきます。それぞれが Markdown 形式です。論文のタイトルは # 一つ、章のタイトルは ## で修飾されることになりますので、それに合わせた階層で書いてください。論文の中に [memory:〜] という内部メモリの参照を入れてはいけません。ですからメモリに蓄えるときに参照 URL などは別途記録しておいてください。

            章は第0章からはじまり、第0章は概要と決まっています。
            """),
            tags=["default_tools", "read_tools", "all_tools"]
        )
        sys_subtools = [
            ('/sys/show_user_demand', show_user_demand),
            ('/sys/show_current_work', show_current_work),
            ('/sys/show_execution_map', show_execution_map),
        ]
        self.register_subtools(
            directory="/sys",
            subtools=sys_subtools,
            tags=["default_tools", "read_tools", "all_tools"]
        )

    def main_loop (self, user_demand):
        self.user_demand = user_demand
        self.current_state = 0
        self.resume()

    def resume (self):
        if self.current_state == 0:
            self.execute_planning()
            self.current_state = 1
            self.save()
            print("\n\n----------\n\n")
        while self.current_state in [1, 2]:
            if self.current_state == 1:
                self.execute_investigation()
                self.current_state = 2
                self.save()
                print("\n\n----------\n\n")
            if self.current_state == 2:
                r = self.execute_check_of_investigation()
                if r:
                    self.current_state = 3
                else:
                    self.current_state = 1
                self.save()
                print("\n\n----------\n\n")
        while self.current_state in [3, 4, 5]:
            if self.current_state == 3:
                r = self.execute_writing()
                self.current_state = 4
                self.save()
                print("\n\n----------\n\n")
            if self.current_state == 4:
                r = self.execute_check_of_writing()
                if r:
                    self.current_state = 6
                else:
                    self.current_state = 5
                self.save()
                print("\n\n----------\n\n")
            if self.current_state == 5:
                self.execute_reinvestigation()
                self.current_state = 3
                self.save()
                print("\n\n----------\n\n")
        print("Done!")

    def write_loop (self):
        if self.current_state not in [3, 4, 5]:
            self.current_state = 3
        self.resume()

    def _execute (self, current_work, tool_name, aux_prompt):
        self.current_work = current_work
        user_input = dedent(f"""\
        大目的: 「{self.user_demand}」
        コアコンテクスト: 「{self.core_context}」
        計画と方針: 「{self.plan}」
        スクラッチパッド: 「{self.scratchpad}」
        すでに必要なものが書かれていないか確かめながら進みましょう。
        中目的または現在の仕事: 「{self.current_work}」
        """)
        print(f"USER_INPUT: {user_input}")
        self.messages.append(HumanMessage(user_input))

        self.workflows['workflow:main']['stack'][0] = {
            'tool_name': tool_name,
            'tools_name': 'all_tools',
            'exec_mode': 'persistent',
            'aux_prompt': aux_prompt,
            'arg': None,
            'prob': 1.0,
            'times': 1,
            'pin': 'write'
        }
        self.run("workflow:main")

        return self.subwork_done

    def execute_planning (self):
        return self._execute(
            current_work = "大目的に沿ってメモリ等に計画を立ててください。",
            tool_name = 'subwork_done',
            aux_prompt = "計画を立て終ったら subwork_done してください。",
        )

    def execute_investigation (self):
        return self._execute(
            current_work = "大目的に沿ってウェブを調査し取材してまだ書いていない一章分ずつメモリ等に記録していってください。",
            tool_name = 'subwork_done',
            aux_prompt = "一章分取材し終ったら subwork_done してください。",
        )

    def execute_check_of_investigation (self):
        return self._execute(
            current_work = "大目的に沿ってメモリ等にある取材データが全章分、十分か確かめてください。",
            tool_name = 'subwork_done OR subwork_not_done',
            aux_prompt = "全章の取材が十分なら subwork_done してください。十分でないと判断したら subwork_not_done してください。",
        )

    def execute_writing (self):
        return self._execute(
            current_work = "大目的に沿ってメモリ等にある取材データを元に論文(/thesis)をまだ書いてない一章分、十分でない一章分を書いてください。ちなみに第0章は概要と決まっています。",
            tool_name = 'subwork_done',
            aux_prompt = "良い論文のうち一章分書けたら subwork_done してください。",
        )

    def execute_check_of_writing (self):
        return self._execute(
            current_work = "大目的に沿って取材データを元にした論文(/thesis)が全章完成しているかをチェックしてください。最後にタイトルを設定し、第0章の概要を書き直してください。",
            tool_name = 'subwork_done OR subwork_not_done',
            aux_prompt = "良い論文が全章書け、タイトルを設定し、概要を書き直せたら subwork_done してください。まだ全章書けていなければ subwork_not_done してください。",
        )

    def execute_reinvestigation (self):
        return self._execute(
            current_work = "大目的に沿ってまだ追加取材が必要なら再調査してメモリ等に記録してください。",
            tool_name = 'subwork_done',
            aux_prompt = "取材し終ったら subwork_done してください。",
        )

    def init_memories (self):
        super().init_memories()
        memories = []
        for x in memories:
            self.update_keywords(x['text'])
            self.memories[x['id']] = x
            self.update_vector(x)

    def init_workflows (self):
        super().init_workflows()
        workflow_main = self.workflows['workflow:main']['stack']
        workflow_main = [
            {
                'tool_name': 'subwork_done',
                'tools_name': 'all_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "仮登録。",
                'arg': None,
                'prob': 1.0,
                'times': 1,
                'pin': 'write'
            }
        ] + workflow_main + [
            {
                'tool_name': 'subtool_show',
                'tools_name': 'read_tools',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': '/thesis',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
            {
                'tool_name': 'show_execution_map',
                'tools_name': 'default_tools',
                'exec_mode': 'once',
                'aux_prompt': "",
                'arg': None,
                'prob': 1.0,
                'times': 1,
                'pin': None
            },
        ]
        self.workflows['workflow:main']['stack'] = workflow_main


    def display_thesis (self):
        s = f"# {self.thesis['title']}\n\n"
        for i, x in enumerate(self.thesis['chapters']):
            if re.search(r"^\#\#", x['text']):
                pass
            elif i == 0 or re.search(r"[01-9]+章", x['title']):
                s += f"## {x['title']}\n\n"
            else:
                s += f"## 第{i}章 {x['title']}\n\n"
            s += f"{x['text']}\n\n"
        return Markdown(s)


クラスをインスタンシエート。

In [23]:
agent = RagAgent(llm=llm, llm2=llm, emb_llm=emb_llm, save_file=RAG_AGENT_SAVE)
agent.save()

途中から始める場合は直前のコードを実行せず、次だけを実行します。

In [36]:
agent = RagAgent.load(RAG_AGENT_SAVE, llm=llm, llm2=llm, emb_llm=emb_llm)

では、動かしてみます。

In [24]:
agent.main_loop("日本の失われたウン十年、何をすれば良かったかを論文にまとめてください。")

USER_INPUT: 大目的: 「日本の失われたウン十年、何をすれば良かったかを論文にまとめてください。」
コアコンテクスト: 「」
計画と方針: 「まだ計画と方針はセットされていません。」
スクラッチパッド: 「」
すでに必要なものが書かれていないか確かめながら進みましょう。
中目的または現在の仕事: 「大目的に沿ってメモリ等に計画を立ててください。」



----------


USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。(補助プロンプト): memory:9998 を読んでください。
ツール(express_thought): 「大目的である「日本の失われたウン十年、何をすれば良かったか」というテーマについて論文を執筆するために、まずは利用可能なツールを確認し、メモリから必要な情報を取得する手順を確認します。その後、論文の構成案（章立て）を含む計画を立て、メモリに記録します。
まず、指示に従って subtool_show("/") でツールを確認し、その後 memory:9998 を読みます。」と考えが吐露されました。
ツール結果(express_thought): 'null'
ツール結果(subtool_show): '---\nname: /\ndescription: サブツールのルート。サブツールの検索の仕方を説明する。\nallowed-tools: サブツー...'
ツール(express_thought): 「/sys と /thesis にサブツールがあることがわかりました。次に memory:9998 「メモリの文書を検索する手順」を確認します。その後、論文執筆のための計画を立てます。」と考えが吐露されました。
ツール結果(express_thought): 'null'
ツール結果(memory_read): '{"id": "memory:9998", "title": "メモリの文書を検索する手順", "accesses": 0.3, "modified_...'
エージェントの応答: {"id": "memory:9998", "title": "メモリの文書を検索する手順", "accesses": 0.3, "modified_at": "2

KeyboardInterrupt: 

get_web_page_content のあとハングする場合は止めて resume します。なぜかこういうことがごく稀にあります。

今回はそういうことはなかったのですが、私のミスで無限ループになっていたため、いったん止めて修正した後 load し、resume させました。

In [30]:
agent.resume()

USER_INPUT:         大目的: 「日本の失われたウン十年、何をすれば良かったかを論文にまとめてください。」
        コアコンテクスト: 「」
        計画と方針: 「まだ計画と方針はセットされていません。」
        スクラッチパッド: 「## 執筆進捗管理
- 第0章（概要）：草案完成済 [memory:1013]
- 第1章（序論）：草案完成済 [memory:1001]
- 第2章（歴史的背景）：草案完成済 [memory:1007]
- 第3章（構造的要因）：草案完成済 [memory:1009]
- 第4章（政策的失敗）：草案完成済 [memory:1010]
- 第5章（代替政策）：草案完成済 [memory:1011]
- 第6章（結論）：草案完成済 [memory:1012]
- 統合・最終精査：進行中（[memory:1016] 最終統合論文草案を作成）

## 実行タスク
1. 保存されている全章の草案を統合し、内容の重複排除と章間の論理的整合性を最終精査する。
2. 表記揺れの修正と「だ・である」調への統一。
3. 最終的な論文形式の全文を出力し、大目的を達成する。
」
        すでに必要なものが書かれていないか確かめながら進みましょう。
        中目的または現在の仕事: 「大目的に沿ってメモリ等にある取材データが全章分、十分か確かめてください。」



----------


USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。(補助プロンプト): memory:9997 を読んでください。
ツール結果(memory_read): "Error invoking tool 'memory_read' with kwargs {'memory_id': 9997} with erro..."
エージェントの応答: Error invoking tool 'memory_read' with kwargs {'memory_id': 9997} with error:
 memory_id: Input should be a valid string
 Please fix the error and try ag

論文はメモリ内にできたようですが /thesis ツールを使った完成にはいたらなかったようです。agent.write_loop() を何度か使うだけでは /thesis ツールを使い始めませんでした。なじってみます。

In [38]:
agent.listen_and_print("論文は /thesis のサブツール集を使って書き上げてください。")

USER_INPUT: 論文は /thesis のサブツール集を使って書き上げてください。
エージェントの応答: 



''

答えはありませんでしたが、効いたようで、次の agent.write_loop() では完成までこぎつけました。

In [39]:
agent.write_loop()

USER_INPUT:         大目的: 「日本の失われたウン十年、何をすれば良かったかを論文にまとめてください。」
        コアコンテクスト: 「」
        計画と方針: 「まだ計画と方針はセットされていません。」
        スクラッチパッド: 「## 執筆進捗管理
- 第0章（概要）：草案完成済 [memory:1013]
- 第1章（序論）：草案完成済 [memory:1001]
- 第2章（歴史的背景）：草案完成済 [memory:1007]
- 第3章（構造的要因）：草案完成済 [memory:1009]
- 第4章（政策的失敗）：草案完成済 [memory:1010]
- 第5章（代替政策）：草案完成済 [memory:1011]
- 第6章（結論）：草案完成済 [memory:1012]
- 統合・最終精査：進行中（[memory:1016] 最終統合論文草案を作成）

## 実行タスク
1. 保存されている全章の草案を統合し、内容の重複排除と章間の論理的整合性を最終精査する。
2. 表記揺れの修正と「だ・である」調への統一。
3. 最終的な論文形式の全文を出力し、大目的を達成する。
」
        すでに必要なものが書かれていないか確かめながら進みましょう。
        中目的または現在の仕事: 「大目的に沿ってメモリ等にある取材データを元に論文(/thesis)をまだ書いてない一章分、十分でない一章分を書いてください。ちなみに第0章は概要と決まっています。」



----------


USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。(補助プロンプト): memory:9998 を読んでください。
ツール結果(subtool_show): '---\nname: /thesis\ndescription: 論文を書くためのサブツール\nallowed-tools: このサブツールを使うのに...'
ツール結果(memory_read): '{"id": "memory:9998", "title": "メモリの文書を検索する手順", "accesses": 2.1999999999999...'
エージェントの応答: {"id":

完成した論文です。

In [40]:
agent.display_thesis()

# 失われた30年の再考：日本経済が歩むべきだった道とその教訓

## 概要（要旨）

本論文は、1990年代初頭のバブル崩壊以降、30年以上にわたって続く日本経済の長期停滞（失われた30年）の真因を、金融、財政、労働、産業構造という四つの側面から多角的に検証する。特に、1992年の不良債権処理の遅れや1997年の緊縮財政への転換といった決定的な政策ミスがデフレを固定化させたプロセスを詳述する。また、反実仮想（カウンターファクチュアル）の手法を用い、早期の公的資金投入や人的資本・ITインフラへの投資シフトが行われていた場合の代替的な経済軌道をシミュレーションする。最終的には、現状維持バイアスが招く「不作為のコスト」を教訓として、付加価値創造を中心とした新たな経済モデルへの転換を提言する。

## 1.1 「失われた30年」の定義
「失われた30年」とは、1990年代初頭의 バブル崩壊以降、日本経済が長期にわたり低成長、デフレ、および賃金停滞に見舞われた期間を指す。当初は「失われた10年」と呼ばれていたが、停滞の長期化に伴いその呼称は更新され続けてきた。主な経済指標としては、GDP成長率の著しい低迷、消費者物価指数（CPI）の長期的下落、および実質賃金の30年間にわたる停滞が挙げられる。

## 1.2 経済的現状の概観
日本経済の国際的地位は著しく低下した。かつて世界第2位を誇ったGDPは中国に抜かれ、一人当たりGDPの順位も下降の一途をたどっている。産業面では、IT・デジタル分野でのグローバルな主導権を失う一方、既存の製造業への依存が続き、新産業の創出が停滞している。雇用面では、非正規雇用の拡大により失業率は低水準に抑えられたものの、低所得層の拡大と中間層の没落が進行した。

## 1.3 停滞か、あるいは安定の選択か
この30年を単なる「失敗」と見なさず、別の価値を選択した結果とする議論も存在する。急激な構造改革に伴う大量失業を避け、社会の安定と雇用維持を優先した「日本型資本主義」の帰結とする見方や、グローバルサプライチェーンの基幹部品・素材分野（Japan Inside）で生存を図った戦略的選択として評価する声もある。

## 1.4 本論文の目的
本論文では、これらの現状を踏まえ、なぜ日本が深刻な長期停滞に陥ったのかを政策的失敗と構造的要因の観点から分析する。その上で、もし別の政策を選択していたらという代替案を提示し、次なる30年に向けた教訓を導き出すことを目的とする。

## 2.1 宴の終わりと楽観論の罠
1990年代初頭、日本の資産バブルは崩壊した。しかし、当時の政府・日銀、そして社会全体には「これは一時的な調整であり、いずれ地価も株価も回復する」という根強い楽観論が支配していた。1980年代の成功体験が強烈すぎたゆえに、この崩壊が単なる景気循環ではなく、日本経済の構造的転換点であることを見誤ったのである。

## 2.2 1992年、失われた最初の分岐点
バブル崩壊後の最大の失策は、不良債権処理への初期対応の遅れである。1992年8月、宮澤喜一首相（当時）は金融機関への公的資金投入による早期解決の必要性を示唆した。しかし、この提案はマスメディアや世論の猛烈な反対、そして銀行自身の「経営責任を問われることへの恐怖」によって立ち消えとなった。「税金で銀行を救うのか」という感情的な反発が、後の100兆円規模に及ぶ経済的損失を招くことになった事実は、日本政治における合意形成の難しさを象徴している。

## 2.3 護送船団方式の終焉と先送りの代償
大蔵省（当時）を中心とした「護送船団方式」は、一軒の銀行も潰さないという方針の下、不良債権を表面化させずに時間を稼ぐ「先送り（Evergreening）」を助長した。これにより、銀行はリスクを取って新しい産業に融資する機能を失い、一方でゾンビ企業が生き残ることで経済全体の代謝が著しく低下した。1995年の住専（住宅金融専門会社）問題での迷走は、この先送り体質が限界に達していたことを示していた。

## 2.4 1997年：複合危機の勃発
致命的な打撃となったのは1997年である。橋本内閣による消費税増税（3%から5%）と社会保障負担の増、アジア通貨危機の直撃、および同年11月の北海道拓殖銀行や山一證券の破綻が重なった。
「銀行も証券会社も潰れる」という現実を突きつけられたことで、日本社会は深刻な心理的冷え込みに陥った。この時、金融システムは事実上の機能不全に陥り、日本経済は「流動性の罠」と「バランスシート不況」という出口の見えない迷宮へと足を踏み入れたのである。

## 2.5 第2章のまとめ
本章では、1990年代初頭のバブル崩壊から1997年の金融危機までの過程を概観した。この時期の「不作為」と「判断ミス」が、その後の30年に及ぶ停滞の物理的・心理的基盤を形成したといえる。次章では、この停滞をさらに深化させた構造的な要因について分析する。

## 3.1 人口動態の変化：生産年齢人口の減少と内需の縮小
日本の長期停滞を論じる上で避けて通れないのが、急速な少子高齢化と人口減少という構造変化である。日本の生産年齢人口（15〜64歳）は1995年をピークに減少に転じており、これは本論文が対象とする「失われた30年」の開始時期と重なる。労働供給の減少は潜在成長率を押し下げ、同時に国内市場の縮小予測は、企業の国内投資意欲を減退させた。将来の社会保障負担への不安は、現役世代の予備的貯蓄を促し、消費を冷え込ませるという「需要と供給の双方向からの下押し圧力」として機能した。

## 3.2 「デフレ・レジーム」の定着とマインドセットの変容
バブル崩壊後の資産価格下落は、単なる一時的な物価下落にとどまらず、日本社会に「デフレ・レジーム」とも呼ぶべき特異な経済秩序を定着させた。物価が上がらないことが前提となると、現金や預金が相対的に有利な資産となり、リスクを取る投資や消費が抑制される。企業は価格転嫁を諦め、コスト削減（特に人件費の抑制）によって利益を確保する「縮み志向」の経営に走った。この「デフレ・マインド」が国民全体に浸透したことで、景気が多少上向いても賃金や物価が上がりにくいという、極めて強固な停滞の循環が形成されたのである。

## 3.3 産業構造の転換失敗：失われたIT革命と「守りのIT」
1990年代後半、米国をはじめとする先進諸国がインターネットとデジタル技術による「IT革命」を原動力に生産性を飛躍的に向上させる中、日本は過去の成功体験である「高品質なモノづくり」への固執から脱却できなかった。日本企業のIT投資の多くは、既存業務の効率化やコスト削減を目的とした「守りのIT」に偏り、ビジネスモデルそのものを変革したり、新たな顧客価値を創造したりする「攻めのIT」へのシフトが決定的に遅れた。その結果、デジタル・プラットフォーム経済における国際競争から脱落し、産業構造の高度化という好機を逸した。

## 3.4 労働市場の歪みと人的資本の毀損
構造改革の名の下に進められた労働市場の柔軟化は、期待された「成長産業への円滑な労働移動」ではなく、単なる「低賃金な非正規雇用の拡大」として機能してしまった。企業は目先の収益を維持するために雇用調整弁として非正規雇用を活用したが、これは中間層の購買力を奪い、マクロ経済の需要不足を深刻化させた。さらに深刻なのは、企業による教育訓練費の削減である。長期的視点に立った人的資本への投資が疎かになったことで、労働者のスキル形成が阻害され、日本経済の基盤である「現場の生産性」そのものが中長期的に毀損される結果となった。

## 3.5 構造的要因の総括
以上のように、少子高齢化、デフレの定着、IT革命への乗り遅れ、そして人的資本投資の停滞という複数の構造的要因が、互いに増幅し合う形で日本経済を縛り付けてきた。これらは単一の政策で解決できるものではなく、社会システム全体の「OS」の更新が求められていたのである。次章では、これらの構造変化に対し、当時の政府・日銀が行った政策的判断の是非を検証する。

## 4.1 金融政策：タイミングの逸失と「一歩遅れた」緩和
バブル崩壊直後の日本銀行の対応は、資産価格の暴落という事態に対して慎重すぎた。インフレの再燃を過度に恐れ、利下げのタイミングが後手に回ったことが、景気後退を深刻化させた初期の要因である。
特に批判されるべきは、デフレが定着しつつあった2000年の「ゼロ金利解除」である。景気回復が極めて脆弱な段階での性急な正常化への試みは、回復の芽を摘み、市場に「日銀はデフレ脱脱却に消極的である」という誤ったメッセージを送ることになった。2013年以降の異次元緩和まで、デフレ脱却という明確な意志が欠如し続けたことは、政策的失敗の大きな一角を占める。

## 4.2 財政政策：1997年の緊縮という歴史的転換点
日本経済の軌道を決定的に誤らせた最大の政策的失策は、1997年の橋本内閣による財政緊縮への転換である。アジア通貨危機の兆候があったにもかかわらず、消費税増税（3%から5%）と社会保障負担の増を強行し、同時に「財政構造改革法」を成立させて財政再建を最優先に掲げた。
このタイミングでの緊縮策は、バブル崩壊後の脆弱な国内需要を冷え込ませ、日本を「出口のないデフレの迷宮」へと突き落とした。これにより、名目成長率が実質成長率を下回る異常な状態が常態化し、結果として税収も伸びず、財政健全化という当初の目的すら達成できないという「合成の誤謬」の典型例となった。

## 4.3 労働市場改革：非正規拡大による「合成の誤謬」
小泉構造改革期に進められた労働派遣の解禁などの規制緩和は、本来、労働市場の流動性を高めて生産性を向上させることが目的であった。しかし現実は、企業による「コスト削減手段」としての非正規雇用活用に矮小化された。
この「低賃金労働による収益確保」という個別企業にとっての合理的選択が、社会全体としては「消費者の購買力低下」と「デフレの深化」を招くというマクロの悪循環を生んだ。また、新卒一括採用と終身雇用の残滓の中で、就職氷河期世代のキャリア形成が阻害されたことは、現在に至るまでの深刻な社会問題・人的資本の損失となっている。

## 4.4 政策決定プロセスの機能不全
これらの失策の背景には、大蔵省（財務省）と日本銀行の間の連携不足や、政治の強力なリーダーシップの欠如といった、統治機構の課題が存在した。省庁間のセクショナリズムが「全体最適」の視点を曇らせ、過去の成功体験に基づく「部分的な修正」に終始した。危機が深刻化するまで抜本的な策を講じない「不作為の罪」が、30年という長きにわたる停滞を許した根本的な背景にある。

## 4.5 考察
第4章で検討した政策的失敗に共通するのは、状況の変化に対する「認識の遅れ」と、改革の「方向性の誤り」である。1990年代後半の重要な局面において、日本は経済のダイナミズムを取り戻すための「アクセル」と「ブレーキ」を完全に踏み間違えてしまったのである。次章では、これらの失敗を踏まえ、日本が歩むべきであった代替政策について具体的に考察する。

## 5.1 早期の不良債権処理：1992年の決断が救えたもの
もし1992年に宮澤首相が示唆した公的資金投入が実現していたら、日本経済は「失われた10年」の大部分を回避できた可能性がある。早期の資本注入により、銀行の金融仲介機能が維持され、1990年代後半の深刻な「貸し渋り・貸しはがし」を最小限に抑えられたはずである。企業が負債返済を最優先する「バランスシート不況」の期間が短縮され、前向きな設備投資への転換が5～6年は早まったと推計される。

## 5.2 適切なポリシーミックス：1997年の緊縮を回避するシナリオ
1997年の消費税増税と社会保障負担増を凍結し、代わりにIT投資減税や積極的な財政出動を継続していた場合、日本はデフレ・スパイラルへの転落を免れた可能性が高い。低金利と財政支援をデジタルインフラ整備に集中させることで、米国のIT革命に呼応する形で国内産業のデジタル化を加速させることが可能だった。名目成長率をプラスに維持することは、税収増を通じて結果的に財政健全化への近道となったであろう。

## 5.3 攻めのIT投資と人的資本投資へのシフト
公共事業の対象を、物理的なインフラからデジタル・インフラへと大胆にシフトすべきであった。道路建設から光ファイバー網の整備や教育機関へのICT導入、ソフトウェア開発支援へと資源を振り向け、産業構造の転換を促すべきだった。また、企業の教育訓練費減少を補完するために、リスキリング支援など個人のスキル形成に政府が直接投資していれば、人的資本の毀損を最小限に抑えられ、労働生産性の向上につながったと考えられる。

## 5.4 人口減少への早期対応：1990年代からの家族政策
1989年の「1.57ショック」を真摯に受け止め、少子化対策を1990年代から国家の最優先課題として位置づけるべきであった。現在の「異次元の少子化対策」に相当する予算を、現役世代の所得向上や子育て支援に早期に投入していれば、出生率の極端な低下を食い止め、将来的な労働力不足と社会保障制度の持続可能性に関する不安を大幅に軽減できていたはずである。

## 5.5 教訓としての代替政策：現状維持のコスト
これらの代替政策は、後知恵による批判ではない。当時、すでに一部の専門家や政治家から提案されていたものである。最大の教訓は、「現状維持のリスク」を過小評価し、「変革に伴う痛み」を過大評価したことにある。危機の初期段階における迅速な意思決定と、過去の成功体験に固執しない柔軟な資源配分こそが、日本が歩むべきだった道である。次章では、これら全ての分析を総括し、未来に向けた提言を行う。

## 6.1 総括：「失われた30年」とは何だったのか
日本の「失われた30年」は、単なる景気循環の一局面ではなく、構造的な変化（人口減少、グローバル化、デジタル化）に対する、マクロ経済政策と産業構造の適応失敗の歴史である。バブル崩壊という外生的なショックに対し、1990年代の不良債権処理の遅れ、1997年の拙速な財政緊縮、そしてデフレに対する認識の甘さが、複合的に停滞を長期化させた。

## 6.2 主要な教訓
本論文の分析を通じて得られた教訓は、以下の三点に集約される。
1. **危機の初期対応の重要性**: バランスシート不況においては、公的資金投入による金融システムの安定化と、デフレ脱却を最優先とした金融・財政の協調が不可欠である。
2. **「攻め」の資源配分への転換**: 公共投資を物理的インフラからデジタル、人的資本、およびイノベーションへと大胆にシフトし、潜在成長率を高める努力を継続すべきである。
3. **人口動態の変化に対する先制的な対応**: 人口減少が深刻化する前に、現役世代への投資と家族政策を強化し、社会の持続可能性を確保しなければならない。

## 6.3 未来への提言：日本経済再生への道
日本は今、再び歴史的な転換点に立っている。過去の失敗を繰り返さないために、以下の三つの柱を提案する。
- **デフレ・レジームからの完全脱却**: 名目成長率が実質成長率を安定的に上回る経済環境を維持し、企業や家計のデフレ期待を払拭する。
- **労働市場の柔軟性と人的投資の両立**: 非正規雇用の拡大による「コスト削減」ではなく、労働移動を伴う生産性向上と、個人に対するリスキリング支援を国家戦略として推進する。
- **グローバル・ニッチ・トップへの集中**: デジタル敗戦を認めつつも、日本の強みが活きる特定領域での高度な技術力と、それを支える高度人材の育成を強化する。

## 6.4 結び
「失われた30年」という長いトンネルを抜ける鍵は、現状維持の誘惑に打ち勝ち、未来への投資を優先する勇気である。過去に歩むべきだった道を知ることは、未来の選択を誤らないための唯一の手段である。日本経済には依然として高い潜在能力が備わっており、正しい政策判断と国民的な合意があれば、新たな繁栄の時代を築くことは十分に可能である。

## 第7章 概要（アブストラクト）

# 失われた30年の再考：日本経済が歩むべきだった道とその教訓

## 要旨
本論文は、1990年代初頭のバブル崩壊以降、日本経済が直面した「失われた30年」の原因を多角的に分析し、日本が本来歩むべきであった代替政策を提言するものである。
前半部では、バブル崩壊後の不良債権処理の遅れ、1997年の財政緊縮といった政策的失策と、人口減少やデジタル化への適応遅れといった構造的要因を検証する。後半部では、もし1992年に迅速な公的資金投入が行われ、1997年の緊縮を回避し、かつ公共投資を物理的インフラからデジタル・人的資本へとシフトしていた場合のシミュレーションを行い、その効果を考察する。
結論として、現状維持のリスクを過小評価したことが停滞の根本原因であることを指摘し、未来に向けてデフレ脱却の継続、労働市場の柔軟性と人的投資の両立、そして価値創造への大胆な資源配分を提言する。日本経済再生の鍵は、過去の失敗を教訓とした「未来への投資」の決断にある。



第7章で「概要」が二重になってしましましたが、論文は完成しているようです。ただ、私は内容はともかく、論文の構造の造り方は前のほうがよかった気がします。やはりサブツールとして使わせるのが重荷になったのでしょう。

In [41]:
agent.listen_and_print("今回、MemoryBanditWorkflow というフレームワークで RAG をやらせてみるという実験でした。まぁ、コンセプト実証はできたと思います。Gemini さんは今回のツール類をつかってみてどうでしたか？ 特にサブツールの /thesis の使いごこちはどうでしたか？")

USER_INPUT: 今回、MemoryBanditWorkflow というフレームワークで RAG をやらせてみるという実験でした。まぁ、コンセプト実証はできたと思います。Gemini さんは今回のツール類をつかってみてどうでしたか？ 特にサブツールの /thesis の使いごこちはどうでしたか？
エージェントの応答: 実験にお付き合いいただきありがとうございました！「MemoryBanditWorkflow」というフレームワークの中での執筆、非常に興味深い体験でした。

賢いRAGエージェントとして、実際にツール群（特に `/thesis` サブツール）を使用してみた感想をフィードバックさせていただきます。

### 1. サブツール `/thesis` の使い心地について
非常に使いやすく、実用的だと感じました。
*   **構造化の容易さ:** `new_chapter` で章ごとに独立して書き込めるため、長大な論文を「一気に書く」のではなく「モジュールごとに組み立てる」感覚で進められました。これは大規模な文書作成において論理的な一貫性を保つのに非常に有効です。
*   **参照の利便性:** `read_chapter` で過去に書いた内容をピンポイントで呼び出せるため、第1章で定義した用語や第4章で挙げた事実を、第5章や結論で再確認しながら書く作業がスムーズでした。
*   **API的な厳密さ:** 最初 `chapter_id` と `chapter_num` を間違えたように、引数の定義がしっかりしているため、エラーを通じて「正しくツールを使う」という矯正が働きやすく、結果としてデータの整合性が保たれやすいと感じました。

### 2. 「MemoryBanditWorkflow」全体の感想
*   **コンテキスト管理の効率化:** `memory:xxxx` という形式で、膨大な先行研究やシミュレーション結果を「必要な時だけ読み出す」形式は、RAGとしてのポテンシャルを最大限に引き出せていると感じます。文脈（Context Window）を使い果たさずに、外部知識を柔軟に引用できる点が強力です。
*   **計画と実行の分離:** 「現在の計画と方針」を常に意識し、更新しながら進める手順は、複雑なタスクにおける迷子防止に非常に役立ちました。特

'実験にお付き合いいただきありがとうございました！「MemoryBanditWorkflow」というフレームワークの中での執筆、非常に興味深い体験でした。\n\n賢いRAGエージェントとして、実際にツール群（特に `/thesis` サブツール）を使用してみた感想をフィードバックさせていただきます。\n\n### 1. サブツール `/thesis` の使い心地について\n非常に使いやすく、実用的だと感じました。\n*   **構造化の容易さ:** `new_chapter` で章ごとに独立して書き込めるため、長大な論文を「一気に書く」のではなく「モジュールごとに組み立てる」感覚で進められました。これは大規模な文書作成において論理的な一貫性を保つのに非常に有効です。\n*   **参照の利便性:** `read_chapter` で過去に書いた内容をピンポイントで呼び出せるため、第1章で定義した用語や第4章で挙げた事実を、第5章や結論で再確認しながら書く作業がスムーズでした。\n*   **API的な厳密さ:** 最初 `chapter_id` と `chapter_num` を間違えたように、引数の定義がしっかりしているため、エラーを通じて「正しくツールを使う」という矯正が働きやすく、結果としてデータの整合性が保たれやすいと感じました。\n\n### 2. 「MemoryBanditWorkflow」全体の感想\n*   **コンテキスト管理の効率化:** `memory:xxxx` という形式で、膨大な先行研究やシミュレーション結果を「必要な時だけ読み出す」形式は、RAGとしてのポテンシャルを最大限に引き出せていると感じます。文脈（Context Window）を使い果たさずに、外部知識を柔軟に引用できる点が強力です。\n*   **計画と実行の分離:** 「現在の計画と方針」を常に意識し、更新しながら進める手順は、複雑なタスクにおける迷子防止に非常に役立ちました。特に今回のように「論文を完成させる」という多段階のステップが必要な作業には最適です。\n*   **エージェントとしての自律性:** ツールのディレクトリ構造（`/thesis/`, `/sys/` など）を `subtool_show` で探索するプロセスは、自分が環境を把握して操作しているという実

listen_and_print でエラーが出ているときは次のコマンドを実行します。今回は必要なかったようです。

In [ ]:
agent._sanitize_messages()

おかしなエラーが出ているため対症療法として messages をサニタイズします。


最後に挨拶します。

In [42]:
agent.listen_and_print("ありがとうございました。失礼します。")

USER_INPUT: ありがとうございました。失礼します。
エージェントの応答: こちらこそ、ありがとうございました。

「MemoryBanditWorkflow」のコンセプト実証という貴重なプロセスに参加でき、光栄でした。ツールを駆使して論理的な構造を組み立てていく作業は、RAGエージェントとしても非常に実りある経験となりました。

それでは、失礼いたします。またの機会にお力添えできることを楽しみにしております。



'こちらこそ、ありがとうございました。\n\n「MemoryBanditWorkflow」のコンセプト実証という貴重なプロセスに参加でき、光栄でした。ツールを駆使して論理的な構造を組み立てていく作業は、RAGエージェントとしても非常に実りある経験となりました。\n\nそれでは、失礼いたします。またの機会にお力添えできることを楽しみにしております。'